In [ ]:
import glob
import math
import numpy as np
from numpy.lib.function_base import meshgrid
import pandas as pd
import pkgutil

from importlib import reload
#syntax: fugi = reload(fugi)

import functions_giammi as fugi

import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.colors import LightSource
%matplotlib inline
#%config InlineBackend.figure_formats=["svg"]
%config InlineBackend.figure_formats=["png"]

import mplhep as hep
from mpl_toolkits.mplot3d import Axes3D
from mpl_toolkits.mplot3d.art3d import Poly3DCollection

import plotly.graph_objects as go
from plotly.subplots import make_subplots
import itertools

plt.style.use(hep.style.ROOT)  #For now ROOT defaults to CMS
#hep.set_style({"font.size":8})
#mpl.rc('font', size=8)

#loads the temperaure cmap like Philipp for both matpplotlib and plotly
cmap_temp, cmap_temp_go, Magma_r, Seismic_r = fugi.customcmaps()

#TFMOx
# path = r'../PYTHON_graphs/DATA/Theory/TFMox-S/enant=S_hel=+1_KE=3.0eV';en="KE=3eV";
# path = r'../PYTHON_graphs/DATA/Theory/TFMox-S/enant=S_hel=+1_KE=4.0eV';en="KE=4eV";
# path = r'../PYTHON_graphs/DATA/Theory/TFMox-S/enant=S_hel=+1_KE=6.1eV';en="KE=6.1eV";
# path = r'../PYTHON_graphs/DATA/Theory/TFMox-S/enant=S_hel=+1_KE=8.2eV';en="KE=8.2eV";
# path = r'../PYTHON_graphs/DATA/Theory/TFMox-S/enant=S_hel=+1_KE=11.5eV';en="KE=11.5eV";

#MOx
path = r'../PYTHON_graphs/DATA/Theory/MOx-R/MoX_CPR_positive';en="MOx_CPR";
path = r'../PYTHON_graphs/DATA/Theory/MOx-R/MoX_CPL_negative';en="MOX_CPL";

#glob has to be sorted (both in unix and windows)
all_files = sorted(glob.glob(path + "/**.dat"))

#if True it computes the 72 MFPAD
MFPAD=True
original=False
std_calculations=False
noprint = True

li = []
cosphi_th = []
test_opening = []
allnames=[]
colnames = ["phi", "cos(theta)", "value"]

#how to load multiple files http://jonathansoma.com/lede/foundations-2017/classes/working-with-many-files/class/,
for filename in all_files:
    df = pd.read_csv(filename, delimiter=r"\s+", names=colnames, header=None)  #r"\s+" is a regex (regular expression)
    #or delim_whitespace=True, it is faste
    temp_name=(filename.replace(" ","")).split("/")[-1].split(".")[0] #adding a column to df with the file name with no spaces and extension
    cosphi_th.append((int(temp_name.split("_")[0]), int(temp_name.split("_")[1])))
    allnames.append(temp_name)
    df["filename"]=temp_name
    df["filename"]=df["filename"].astype("category")
    li.append(df) #a unique DataFrame

frame = pd.concat(li, axis=0)
#this commmand avoids 1_1 followed by 1_10: semantic order
frame_srt = frame.groupby("filename",sort=False)
frame_set = frame.set_index("filename") #build a multiindex using the categories of filename

In [ ]:
#test initial plotting
counts_1_1 = frame.loc[frame["filename"] == "1_1"].iloc[:,2].to_numpy() #values
counts_1_1 = counts_1_1.reshape(200,100) #reshape(rows=x,columns=y)

print("for reshaping: number of ROWS phi (x) = ", len(counts_1_1[:,1]), "number of COLUMNS cos (y)  = ", len(counts_1_1[1,:]), "type ", type(counts_1_1))

In [ ]:
#Single 2D maps from pandas without tranforming into numpy
frame.loc[frame["filename"] == "1_1"].plot.hexbin(x="phi", y="cos(theta)", C="value", sharex=False, gridsize=50)

In [ ]:
"""
##############################################NOTE####################################################
in elements tag:
cosphi_th=([[ 1,  1],   cosphi_th_cos=([[ 6,  1],       cosphi_phi=([[ 6,  1],      
            [ 1,  2],                   [ 6,  2],                    [ 5,  1],                   
            [ 1,  3],                   [ 6,  3],                    [ 4,  1],
            [ 1,  4],                   [ 6,  4],                    [ 3,  1],
            [ 1,  5],                   [ 6,  5],                    [ 2,  1],                   
            [ 1,  6],                   [ 6,  6],                    [ 1,  1],

in angles tag:
cosphi_adj_th=([[ 1,  -180],   cosphi_adj_cos=([[ -1,  -180],       cosphi_adj_phi=([[ -1,   -180],      
                [ 1,  -147],                    [ -1,  -147],                        [ -0.8, -180],                   
                [ 1,  -114],                    [ -1,  -114],                        [ -0.3, -180],
                [ 1,  -81],                     [ -1,  -81],                         [ 0.3,  -180],
                [ 1,  -49],                     [ -1,  -49],                         [ 0.8,  -180],                   
                [ 1,  -16],                     [ -1,  -16],                         [ 1,    -180],

cosphi_th has to be used JUST with counts_stemp!!

b1 maps cordinates. YOU SHOULD USE THIS AS ROTATION INPUT

list(zip(xgo_phi,ygo_phi))             list(zip(xgo_cos,ygo_cos))
[(-180.0, 1.0),                        [(-180.0, 1.0),
 (-180.0, 0.809),                       (-147.273, 1.0),
 (-180.0, 0.309),                       (-114.545, 1.0),
 (-180.0, -0.309),                      (-81.818, 1.0),
 (-180.0, -0.809),                      (-49.091, 1.0),
 (-180.0, -1.0),                        (-16.364, 1.0),
##############################################NOTE####################################################
######################################################################################################
                                    PHI - CTHETEA ELECTRON 
######################################################################################################
(alpha and beta in Philipp convention)
REMEBER: in Philipp simulations cos(theta)=[1,-1] becase theta=[0,np.pi]
SORTED = cos(theta)=[-1,1] IS NEVER USED for electrons coordinates
#NOTE: the following phi and ctheta are not unique: len=20000

"""
#SORTED FULL **cos(theta)_el = [1,-1]**, phi_el = [-179.1, 179.1]
#len = 200000
phi_full_sorted=frame_srt.get_group("1_1").sort_values(by=["phi","cos(theta)"])["phi"] 
phi_rad_sorted = phi_full_sorted * np.pi/180.

ctheta_full_sorted=frame_srt.get_group("1_1").sort_values(by=["phi","cos(theta)"])["cos(theta)"]
theta_rad_sorted = np.arccos(ctheta_full_sorted)
theta_full = theta_rad_sorted*180./np.pi

#SORTED unique **cos(theta)_el = [1,-1]**, phi_el = [-179.1, 179.1]  
#len(phiMs) = 200, len(cosM) = 100
phiM=frame_srt.get_group("1_1").sort_values(by=["phi","cos(theta)"])["phi"].unique()
cosM=frame_srt.get_group("1_1").sort_values(by=["phi","cos(theta)"])["cos(theta)"].unique()

#UNSORTED FULL phi_el = [-179.1, 179.1]
#len = 200000
#phi_full = frame.loc[frame["filename"] == "1_1"].iloc[:,0].to_numpy()
#phi_rad = phi_full * np.pi/180.

#UNSORTED FULL cos(theta)_el = [-1,1]
#len = 200000
#ctheta_full = frame.loc[frame["filename"] == "1_1"].iloc[:,1].to_numpy()
#theta_rad = np.arccos(ctheta_full)
#theta_full = theta_rad*180./np.pi

#UNSORTED unique cos(theta)_el = [-1,1], phi_el = [-179.1, 179.1]
#len(phiMu) = 200, len(cosMu) = 100
#NOTE      np.any(phiM == phiMu) -> TRUE
#     np.any(cosM == np.flip(cosMu)) -> TRUE
phiMu = frame.loc[frame["filename"] == "1_1"].iloc[:,0].unique() #phi_el_u = [-179.1, 179.1]
cosMu = frame.loc[frame["filename"] == "1_1"].iloc[:,1].unique() #cos(theta)_el_u = [-1,1]

#NOTE: for np.MESHGRID:
#for the 2D case with inputs of length M and N, the outputs are of shape (N, M) for ‘xy’ indexing and (M, N) for ‘ij’ indexing. Cartesia "xy" by default
#REMEBER: np.any(phiM == phiMs) -> TRUE

#This is the right one to use to interpolate if the MFPAD IS SORTED
#because cos(theta)_el = [1,-1]
phiMM, cosMM = np.meshgrid(phiM, cosM)
phiMM_rad, thetaMM_rad = np.meshgrid(phiM*np.pi/180., np.arccos(cosM))

#because **cos(theta)_el = [1,-1]**
#phiMMu, cosMMu = np.meshgrid(phiMu, cosMu)
#phiMMu_rad, thetaMMu_rad = np.meshgrid(phiMu*np.pi/180., np.arccos(cosMu))

print("pihM ", phiM.shape, "cosM ", cosM.shape)
print("cosM[0] ", cosM[0])
print("pihMM ", phiMM.shape, "cosMM ", cosMM.shape)
print(type(phiM), type(phiMM))

In [ ]:
"""
##########################b1 map x,y coordiantes###############################
PHI - CTHETA PHOTON with respect to the LF (in sperical coordinates)
NOTE the go coordiantes are organised accordinf to the SORTED MFPAD, therefore cos(theta)_PHOTON=[-1,1]
REDUCED=TRUE => ADJUSTED TO EXPERIMENT: -0.835,0.835,-165,165
REDUCED=FALSE => ADJUSTED TO THEORY: -1,1,-180,180
"""
fugi = reload(fugi)
#the std goes cos(theta)_photon [1,-1] therefore uses cos_th_adj
#the sorted cos has cos(theta)_photon [1,-1] therefoere uses cos_th_adj_cflip
#deprecated: it is linearly disributed in cos(theta), but the real data are linear in theta!
#cosphi_th_adj=np.around(np.array(list(itertools.product(np.linspace(1.,-1.,6).tolist(),np.linspace(-180,180,12).tolist()))),3)

#cosphi_th_adj_exp = np.around(np.array(list(itertools.product(np.linspace(0.835,-0.835,6).tolist(),np.linspace(-165,165,12).tolist()))),3)
#cosphi_th_adj_exp_cflip = np.around(np.array(list(itertools.product(np.linspace(-0.835,0.835,6).tolist(),np.linspace(-165,165,12).tolist()))),3)

#half of 30 deg in radiants 
#cos_lin=False; limits=[np.pi-0.261799388, 0.261799388, -180., 180.] #linear in theta [np.pi,0]
cos_lin=True; limits=[-1+1/6, 1-1/6, -180+33/2, 180-33/2] #linear in ctheta [-1,1], default with Philipp
#cos_lin=True; limits=[-0.95,0.95, -175., 175.] #linear in ctheta [-1,1], default with Philipp
#cos_lin=True; limits=[-1,1,-180,180] #linear in ctheta reduced experiment

#probably linear in theta just in the theory
xgo_phi,ygo_phi,cosphi_adj_phi = fugi.create_gocoords(a=0, cos_lin=cos_lin, limits=limits, source=True)
xgo_cos,ygo_cos,cosphi_adj_cos = fugi.create_gocoords(a=1, cos_lin=cos_lin, limits=limits, source=True)
xgo_cos_flip,ygo_cos_flip,cosphi_adj_cos_flip = fugi.create_gocoords(a=2, cos_lin=cos_lin, limits=limits, source=True)
xgo_phi_flip,ygo_phi_flip,cosphi_adj_phi_flip = fugi.create_gocoords(a=3, cos_lin=cos_lin, limits=limits, source=True)

#NOTE to preserve the scanning along cos(theta)_phot or phi_phot meshgrid is needed. unique and zip do not work!
xgo_uniq=np.unique(np.array(xgo_cos));ygo_uniq=np.unique(np.array(ygo_cos))
xgo_flip_uniq=np.unique(np.array(xgo_cos_flip));ygo_flip_uniq=np.unique(np.array(ygo_cos_flip))

#list(zip(xgo_phi,ygo_phi))
"""
Creation of DataFrame to find the right element according to rotation
isuefull fot the 3d representation
NOTE allnames are UNSORTED: provide cosphi_df with cos(thea) = [1,-1] sorted by cos(theta)
NOTE all the coordinates are FULL!! this is iportant for rotation

according to automatic_72_CPR.f90 from philipp [1,6]: is from [0,P1] -> [cos(0), cos(PHI)] = [1,-1]
according to automatic_72_CPR.f90 from philipp [1,12]: is from [-PI,PI]

allnames  cos(theta)   phi
1_1       1            -180 
1_2       1            -147 
"""
#NOTE cosphi_th_adj comes from manual creation cos [1:-1], phi[-180;180], but same order as original
dfind=pd.DataFrame(cosphi_adj_cos_flip, columns=["ctheta","phi"])
dfind["item"]=allnames

#Sorting the label of the element according to the numerical value of ctheta and phi
cosphi_th_std=(dfind.sort_values(by=["ctheta", "phi"], ascending=[False,True]))["item"].values
cosphi_th_cos=(dfind.sort_values(by=["ctheta", "phi"]))["item"].values
cosphi_th_phi_cflip=(dfind.sort_values(by=["phi","ctheta"],ascending=[True,False]))["item"].values
cosphi_th_phi=(dfind.sort_values(by=["phi","ctheta"]))["item"].values


print("(xgo (phi), ygo (ctheta)) \n", list(zip(xgo_cos,ygo_cos))[:6])
print("(xgo flip (phi), ygo flip (ctheta)) \n", list(zip(xgo_cos_flip,ygo_cos_flip))[:6])

print("\n Default from the theory is 1.")
print("\n 1. cosphi_adj_cos_flip  \n", cosphi_adj_cos_flip[:6], "\n cospi_th_std \n", cosphi_th_std[:6], "\n")
print("\n 2. cosphi_adj_cos \n", cosphi_adj_cos[:6], "\n cospi_th_cos \n", cosphi_th_cos[:6], "\n")
print("\n 3. cosphi_adj_phi \n", cosphi_adj_phi[:6], "\n cospi_th_phi \n", cosphi_th_phi[:6], "\n")
print("\n 4. cosphi_adj_phi_flip \n", cosphi_adj_phi_flip[:6], "\n cospi_th_phi_flip \n", cosphi_th_phi_cflip[:6], "\n")

In [ ]:
#find the photon rotation values of a specific set of rotation angles
#where b1 is max or min!
dfind.sort_values(by=["phi", "ctheta"])
dfind.loc[dfind["item"] == "4_7"]

In [ ]:
fig1, ax = plt.subplots(constrained_layout=True)

cs = ax.contourf(phiMu,cosMu,counts_1_1.T, levels=20, extent=[phiMu[0],phiMu[-1],cosMu[0],cosMu[-1]], vmin=0.0001)
#cs, ax[2]=fugi.plot_interpolation(phiM,cosM,MFPAD_S_cos_LF[71].T,ax=ax[2])

ax.set_xlabel('\u03C6 [DEG]')
ax.set_ylabel('cos\u03D1 [adm]')
ax.set_title("MFPAD S enanant 1_1")
fig1.colorbar(cs)

In [ ]:
frame_srt.head()

In [ ]:
#convert the counts into a 2D np array and sort according to cos and phi
#NOTE: theta is along the z_MF and phi is atan(x_MF,y_MF)
fugi = reload(fugi)
counts_temp=[] #standard cos sorted S enantiomer
check_file=[] #standard cos sorted S enantiomer
MFPAD_R_std=[]
MFPAD_R_phi=[]

#NOTE the values of each MFPAD is espressend with phi[-180,180], cos(theta)[1,-1] for ELECTRONS
#counts_temp: in photon_coordiantes cos(theta)=[-1,1], phi=[-180,180] 1_1, 1_2, 1_3 ...
#in electron coordinates cos(theta)_el=[1,-1], phi_el=[-180,180]
for group in frame_srt:
    counts_temp.append(frame_srt.get_group(group[0]).sort_values(by=["phi","cos(theta)"])["value"].values)
    check_file.append(frame_srt.get_group(group[0]).sort_values(by=["phi","cos(theta)"])["filename"])
    if group[0] == "1_1":
        phi_full_sorted_check=np.array(frame_srt.get_group(group[0]).sort_values(by=["phi","cos(theta)"])["phi"].values)
        ctheta_full_sorted_check=np.array(frame_srt.get_group(group[0]).sort_values(by=["phi","cos(theta)"])["cos(theta)"].values)

#counts_temp: in photon_coordinates as cosphi_th_std: 
#1_1', '1_2', '1_3', '1_4', '1_5', '1_6', '1_7', '1_8' 
MFPAD_S_std=np.array(counts_temp).reshape(72,200,100)

#MFPAD_S_cos: in photon_coordinates as cosphi_th_std 
#OUTPUT a=1 : 6_1', '6_2', '6_3', '6_4', '6_5', '6_6', '6_7', '6_8'
MFPAD_S_cos=fugi.sorting_array(MFPAD_S_std, theory=True, items=cosphi_th_std, a=1)

#MFPAD_S_phi: in photon_coordiantes as cosphi_th_std 
#OUTPUT a=2 : 6_1', '5_1', '4_1', '3_1', '2_1', '1_1', '6_2', '5_2'
MFPAD_S_phi=fugi.sorting_array(MFPAD_S_std, theory=True, items=cosphi_th_std, a=2)

print("order: phi == phi_check ",np.all(phi_full_sorted == phi_full_sorted_check))
print("order: phi == phi_check ",np.all(ctheta_full_sorted == ctheta_full_sorted_check))
#these are all false because of the cos(theta)_photon invertion
print("order: original == phi_ph ",np.all(counts_temp == MFPAD_S_phi))
print("order: original == cos_ph ",np.all(counts_temp == MFPAD_S_cos))

"""
Whith this sorting:
MFPAD_S_phi == MFPAD_S_cos gives True for the first and the last element
MFPAD_S_phi or MFPAD_S_phi == counts_temp are False, because the cos(theta) PHOTON is changed to [-1,1] from the original [1,-1]
"""
#integration of MFPAD onto cos or phi for S enantiomer IN MF
cos_projthS_phi=fugi.projection(MFPAD_S_phi,alongaxis=0)
phi_projthS_phi=fugi.projection(MFPAD_S_phi,alongaxis=1)

#flipping for R enantiomer and sorting: JUST ALONG COS(theta) -> axis=1
for el in MFPAD_S_std:
    MFPAD_R_std.append(np.flip(el,axis=1))
    #MFPAD_R_cos.append(np.flip(MFPAD_R_cos, axis=1))

for el in MFPAD_S_phi:
    MFPAD_R_phi.append(np.flip(el,axis=1))
    #MFPAD_R_cos.append(np.flip(MFPAD_R_cos, axis=1))

#integration of of MFPAD onto cos or phi for R enantiomer
cos_projthR_phi=fugi.projection(MFPAD_R_phi, alongaxis=0)
phi_projthR_phi=fugi.projection(MFPAD_R_phi, alongaxis=1)

#Sum in indepedent of the sorting, it is a total sum:
#MFPAD_R_phi_sum is useless!
#np.any(MFPAD_S_cos_sum == MFPAD_S_phi_sum) TRUE
MFPAD_S_phi_sum=np.sum(MFPAD_S_phi,axis=0)
MFPAD_R_phi_sum=np.sum(MFPAD_R_phi,axis=0)
#MFPAD_R_phi_sum=np.sum(MFPAD_R_phi,axis=0)

print(np.array(MFPAD_S_phi).shape)
print(np.array(MFPAD_R_phi).shape)
print(np.array(cos_projthS_phi).shape)
print(np.array(cos_projthR_phi).shape)

for i, eli in enumerate(MFPAD_S_std):
    for j, elj in enumerate(MFPAD_S_phi):
        if np.all(eli==elj):
            print(i,j)


fugi = reload(fugi)
orig,debug,test=fugi.sorting_array(MFPAD_S_std, theory=True, items=cosphi_th_std, a=1, DEBUG=True)

np.array(check_file)[60]

In [ ]:
################norm might not be needed: already normalised by Philipp ######################
#MFPAD_S_cos_norm=fugi.normalise_matrix(MFPAD_S_cos,normtype=2)
#MFPAD_R_cos_norm=fugi.normalise_matrix(MFPAD_R_cos,normtype=2)

#MFPAD_S_cos_norm=np.array(MFPAD_S_cos_norm)
#MFPAD_R_cos_norm=np.array(MFPAD_R_cos_norm)

#MFPAD_S_cos_norm_sum=np.sum(MFPAD_S_cos_norm,axis=0)
#MFPAD_R_cos_norm_sum=np.sum(MFPAD_R_cos_norm,axis=0)
###############################################################################

#//////////////////////////// mPECD \\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\
#up to now pretty useless
#mPECD_cos=np.divide(np.subtract(MFPAD_R_cos,MFPAD_S_cos),np.add(MFPAD_R_cos,MFPAD_S_cos))
#mPECD_phi=np.divide(np.subtract(MFPAD_R_phi,MFPAD_S_phi),np.add(MFPAD_R_phi,MFPAD_S_phi))

#//////////////////////////// cos theta PECD \\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\
"""
The difference btw cos and phi lays in whitch xgo coordiantes to use when plotting the b1
The error computation is somehow unconventional, but let's normalise the error with the square root of all counts
NOTE: the order (R-S or S-R) seems to have no influence on b1 map. ?? to verify
"""
#cPECD_cos =  np.divide(np.subtract(cos_projthR_cos,cos_projthS_cos),np.add(cos_projthR_cos,cos_projthS_cos))
#errPECD_cos = fugi.error_calc(cos_projthR_cos,cos_projthS_cos)

cPECD_phi =  np.divide(np.subtract(cos_projthR_phi,cos_projthS_phi),np.add(cos_projthR_phi,cos_projthS_phi))
cPECD_phi =  np.divide(np.subtract(cos_projthS_phi,cos_projthR_phi),np.add(cos_projthR_phi,cos_projthS_phi)) ###TEST
errPECD_phi = fugi.error_calc(cos_projthR_phi,cos_projthS_phi)

#//////////////////////////// CONTRAST MAP \\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\
#it is a sum therefore not influenced by the sorting
mPECD_phi_sum=np.divide(np.subtract(MFPAD_R_phi_sum,MFPAD_S_phi_sum),np.add(MFPAD_R_phi_sum,MFPAD_S_phi_sum))

##rotation of the photon vector
b=[col[0] for col in cosphi_adj_phi]
c=np.arccos(b)-20*np.pi/180
d=np.cos(c)
a=[col[1]+10 for col in cosphi_adj_phi]
cosphi_adj_phi=list(zip(d,a))
cosphi_adj_phi

In [ ]:
#exported to pe brocessed in 3D
%store MFPAD_S_std
%store theta_rad_sorted
%store phi_rad_sorted
%store phiM
%store cosM
%store phiMM
%store cosMM

In [ ]:
fugi = reload(fugi)
MFPAD_R_std_LF=[]
MFPAD_R_phi_LF=[]
MFPAD_R_phi_opt_LF=[]
MFPAD_R_phi_rot=[]

test=[]
#rotation of the MFPAD according to the photon rotation and triangulating the result on a ADJUSTED meshgrid phiMM, cosMM:
#####SET THE CONVENTION! what works is convention 1 with option 2. AND 3. in fugi.rot3d_MFPAD#####

print("Performing rotation of MFPAD according to photon coordinates")
#default: s=0.0001
MFPAD_S_std_LF,cos_S_std_LF,phi_S_std_LF=fugi.rot3d_MFPAD(MFPAD_S_std,theta_rad_sorted,phi_rad_sorted,cosphi_adj_cos_flip,phiM,cosM,convention=1,s=0.001)
MFPAD_S_phi_LF,cos_S_phi_LF,phi_S_phi_LF=fugi.rot3d_MFPAD(MFPAD_S_phi,theta_rad_sorted,phi_rad_sorted,cosphi_adj_phi,phiM,cosM,convention=1,s=0.001)
print("Done.")

#flipping sorted S enantiomer for R enantiomer: JUST ALONG COS(theta)_el -> axis=1
for el in MFPAD_S_std_LF:
    MFPAD_R_std_LF.append(np.flip(el, axis=1))

for el in MFPAD_S_phi_LF:
    MFPAD_R_phi_LF.append(np.flip(el, axis=1))

#optimization1_totMFPAD convention 1
#NOTE the angles a,b,c are according to the YPR convention: USE THE OPPOSITE OF THE RESULTS a=-phi, b=-theta
#a=0.5 ;b=1.5 ;c=-2.875; CH = "CH11" #CH11
a=4. ;b=10. ;c=4.5; CH = "CH11" #old
#a=50. ;b=30. ;c=95.; CH = "CH9"

first=[];second=[];third=np.ones(72)*c

for el in cosphi_adj_phi:
    check1=np.arccos(el[0])+b*np.pi/180.
    if check1<0.:
        first.append(np.cos(check1-np.pi))
    elif check1>np.pi:
        first.append(np.cos(check1+np.pi))
    else:
        first.append(np.cos(check1))

    check2=el[1]+a
    if check2<-180.:
        second.append(check2+360.)
    elif check2>180.:
        second.append(check2-360.)
    else:
        second.append(check2)

cosphi_adj_phi_opt=np.column_stack((first,second,third))
cosphi_adj_phi_rot=np.column_stack((np.ones(72)*np.cos(b*np.pi/180.),np.ones(72)*a,np.ones(72)*c))

print("Performing rotation of MFPAD for optimization")
#!check the Euler angle definiton in fuctions_giammi.py line 924/925. The cosphi_adj_phi_rot should be b,a,c ?
MFPAD_S_phi_opt_LF,cos_S_phi_LF,phi_S_phi_LF=fugi.rot3d_MFPAD(MFPAD_S_phi,theta_rad_sorted,phi_rad_sorted,cosphi_adj_phi_opt,phiM,cosM,convention=1,s=0.01)
MFPAD_S_phi_rot,dummy1,dummy2=fugi.rot3d_MFPAD(MFPAD_S_phi,theta_rad_sorted,phi_rad_sorted,cosphi_adj_phi_rot,phiM,cosM,convention=1,s=0.01)
print("Done.")

for el in MFPAD_S_phi_opt_LF:
    MFPAD_R_phi_opt_LF.append(np.flip(el, axis=1))

for el in MFPAD_S_phi_rot:
    MFPAD_R_phi_rot.append(np.flip(el, axis=1))

#test to flip the cos(theta)_photon, not just the cos(theta)_el
#MFPAD_R_std_LF=np.flip(np.array(MFPAD_R_std_LF).reshape(6,12,-1),axis=0).reshape(72,200,100)
#MFPAD_R_cos_LF=np.flip(np.array(MFPAD_R_cos_LF).reshape(6,12,-1),axis=0).reshape(72,200,100)

#integration of MFPAD onto cos  or phi for S enantiomer
#!after the change in the shape of rotated MFPAD, check if the alongaxis are atill ok, could be inverted!
cosx_S_std_LF=fugi.projection(MFPAD_S_std_LF, alongaxis=0)
cosx_S_phi_LF=fugi.projection(MFPAD_S_phi_LF, alongaxis=0)
cosx_S_phi_opt_LF=fugi.projection(MFPAD_S_phi_opt_LF, alongaxis=0)

MFPAD_R_phi_LF_sum=np.sum(MFPAD_R_phi_LF,axis=0)

phi_projthS_std_LF=fugi.projection(MFPAD_S_std_LF, alongaxis=1)
phi_projthS_phi_LF=fugi.projection(MFPAD_S_phi_LF, alongaxis=1)

cosx_R_std_LF=fugi.projection(MFPAD_R_std_LF, alongaxis=0)
cosx_R_phi_LF=fugi.projection(MFPAD_R_phi_LF, alongaxis=0)
cosx_R_phi_opt_LF=fugi.projection(MFPAD_R_phi_opt_LF, alongaxis=0)

phi_projthR_std_LF=fugi.projection(MFPAD_R_std_LF, alongaxis=1)
phi_projthR_phi_LF=fugi.projection(MFPAD_R_phi_LF, alongaxis=1)

#Sum in indepedent of the sorting, it is a total sum
#np.any(MFPAD_S_cos_LF_sum == MFPAD_S_phi_LF_sum) TRUE
MFPAD_S_std_LF_sum=np.sum(MFPAD_S_std_LF,axis=0)
MFPAD_S_phi_LF_sum=np.sum(MFPAD_S_phi_LF,axis=0)
MFPAD_R_phi_LF_sum=np.sum(MFPAD_R_phi_LF,axis=0)
MFPAD_R_phi_opt_LF_sum=np.sum(MFPAD_R_phi_opt_LF,axis=0)

#NOTE rot therefore just optimized, NOT LF!
MFPAD_S_phi_rot_sum=np.nan_to_num(np.sum(MFPAD_S_phi_rot,axis=0))
MFPAD_R_phi_rot_sum=np.nan_to_num(np.sum(MFPAD_R_phi_rot,axis=0))

##//////////////////////////// cos theta PECD \\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\
"""
The difference btw cos and phi lays in whitch xgo coordiantes to use when plotting the b1
The error computation is somehow unconventional, but let's normalise the error with the square root of all counts
NOTE: the order (R-S or S-R) seems to have no influence on b1 map. ?? to verify
"""

cPECD_std_LF_rnd = np.nan_to_num(np.divide(np.subtract(cosx_R_std_LF,cosx_S_std_LF),np.add(cosx_R_std_LF,cosx_S_std_LF)))
errPECD_std_LF_rnd = np.nan_to_num(np.sqrt(1/(cosx_R_std_LF+cosx_S_std_LF)))

#cPECD_phi_LF_rnd = np.nan_to_num(np.divide(np.subtract(cosx_R_phi_LF,cosx_S_phi_LF),np.add(cosx_R_phi_LF,cosx_S_phi_LF)))
cPECD_phi_LF_rnd = np.nan_to_num(np.divide(np.subtract(cosx_S_phi_LF,cosx_R_phi_LF),np.add(cosx_S_phi_LF,cosx_R_phi_LF))) #TEST
errPECD_phi_LF_rnd = np.nan_to_num(np.sqrt(1/(cosx_R_phi_LF+cosx_S_phi_LF)))

#opt because fully rotated + corrected
cPECD_phi_opt_LF_rnd = np.nan_to_num(np.divide(np.subtract(cosx_R_phi_opt_LF,cosx_S_phi_opt_LF),np.add(cosx_R_phi_opt_LF,cosx_S_phi_opt_LF)))
errPECD_phi_LF_rnd = np.nan_to_num(np.sqrt(1/(cosx_R_phi_opt_LF+cosx_S_phi_opt_LF)))

#//////////////////////////// CONTRAST MAP \\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\
#it is a sum therefore NOT influenced by the sorting
#mPECD_LF_phi_sum=np.divide(np.subtract(MFPAD_R_phi_LF_sum,MFPAD_S_phi_LF_sum),np.add(MFPAD_R_phi_LF_sum,MFPAD_S_phi_LF_sum))
mPECD_LF_phi_sum=np.divide(np.subtract(MFPAD_S_phi_LF_sum,MFPAD_R_phi_LF_sum),np.add(MFPAD_S_phi_LF_sum,MFPAD_R_phi_LF_sum)) #TEST

#NOTE rot therefore just optimized, NOT LF!
mPECD_phi_rot_sum=np.nan_to_num(np.divide(np.subtract(MFPAD_R_phi_rot_sum,MFPAD_S_phi_rot_sum),np.add(MFPAD_R_phi_rot_sum,MFPAD_S_phi_rot_sum)))

In [ ]:
plt.plot(cosphi_adj_phi_opt[0:10,:1])
plt.plot(cosphi_adj_phi[0:10,:1])

In [ ]:
#//////////////////////////// PECD \\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\
temp_PECD_R=np.empty([12,6,20000])
temp_PECD_S=np.empty([12,6,20000])
temp_PECD_S_LF=np.empty([12,6,20000])
#cPECD is RS not SR!
temp_cPECD_std=np.empty([12,6,100]); temp_cPECD_phi=np.empty([12,6,100]); temp_cPECD_opt=np.empty([12,6,100]); 
temp_cPECD_err_std=np.empty([12,6,100]); temp_cPECD_err_phi=np.empty([12,6,100]); temp_cPECD_err_opt=np.empty([12,6,100])

for j in range(12):
    for i in range(6):
        if j>5:
            temp_PECD_R[j,i]=np.divide(np.subtract(np.array(MFPAD_R_phi).reshape(12,6,-1)[j,i],np.array(MFPAD_R_phi).reshape(12,6,-1)[j-6,5-i]),np.add(np.array(MFPAD_R_phi).reshape(12,6,-1)[j,i],np.array(MFPAD_R_phi).reshape(12,6,-1)[j-6,5-i]))
            temp_PECD_S[j,i]=np.divide(np.subtract(np.array(MFPAD_S_phi).reshape(12,6,-1)[j,i],np.array(MFPAD_S_phi).reshape(12,6,-1)[j-6,5-i]),np.add(np.array(MFPAD_S_phi).reshape(12,6,-1)[j,i],np.array(MFPAD_S_phi).reshape(12,6,-1)[j-6,5-i]))
            temp_PECD_S_LF[j,i]=np.divide(np.subtract(np.array(MFPAD_S_phi_LF).reshape(12,6,-1)[j,i],np.array(MFPAD_S_phi_LF).reshape(12,6,-1)[j-6,5-i]),np.add(np.array(MFPAD_S_phi_LF).reshape(12,6,-1)[j,i],np.array(MFPAD_S_phi_LF).reshape(12,6,-1)[j-6,5-i]))
            temp_cPECD_std[j,i]=np.divide(np.subtract(cosx_R_std_LF.reshape(12,6,-1)[j,i],cosx_R_std_LF.reshape(12,6,-1)[j-6,5-i]),np.add(cosx_R_std_LF.reshape(12,6,-1)[j,i],cosx_R_std_LF.reshape(12,6,-1)[j-6,5-i]))
            temp_cPECD_err_std[j,i]=fugi.error_calc(cosx_R_std_LF.reshape(12,6,-1)[j,i],cosx_R_std_LF.reshape(12,6,-1)[j-6,5-i])
            temp_cPECD_phi[j,i]=np.divide(np.subtract(cosx_R_phi_LF.reshape(12,6,-1)[j,i],cosx_R_phi_LF.reshape(12,6,-1)[j-6,5-i]),np.add(cosx_R_phi_LF.reshape(12,6,-1)[j,i],cosx_R_phi_LF.reshape(12,6,-1)[j-6,5-i]))
            temp_cPECD_err_phi[j,i]=fugi.error_calc(cosx_R_phi_LF.reshape(12,6,-1)[j,i],cosx_R_phi_LF.reshape(12,6,-1)[j-6,5-i])
            temp_cPECD_opt[j,i]=np.divide(np.subtract(cosx_R_phi_opt_LF.reshape(12,6,-1)[j,i],cosx_R_phi_opt_LF.reshape(12,6,-1)[j-6,5-i]),np.add(cosx_R_phi_opt_LF.reshape(12,6,-1)[j,i],cosx_R_phi_opt_LF.reshape(12,6,-1)[j-6,5-i]))
            temp_cPECD_err_opt[j,i]=fugi.error_calc(cosx_R_phi_opt_LF.reshape(12,6,-1)[j,i],cosx_R_phi_opt_LF.reshape(12,6,-1)[j-6,5-i])       
       
        else:
            temp_PECD_R[j,i]=np.divide(np.subtract(np.array(MFPAD_R_phi).reshape(12,6,-1)[j,i],np.array(MFPAD_R_phi).reshape(12,6,-1)[6+j,5-i]),np.add(np.array(MFPAD_R_phi).reshape(12,6,-1)[j,i],np.array(MFPAD_R_phi).reshape(12,6,-1)[6+j,5-i]))
            temp_PECD_S[j,i]=np.divide(np.subtract(np.array(MFPAD_S_phi).reshape(12,6,-1)[j,i],np.array(MFPAD_S_phi).reshape(12,6,-1)[6+j,5-i]),np.add(np.array(MFPAD_S_phi).reshape(12,6,-1)[j,i],np.array(MFPAD_S_phi).reshape(12,6,-1)[6+j,5-i]))
            temp_PECD_S_LF[j,i]=np.divide(np.subtract(np.array(MFPAD_S_phi_LF).reshape(12,6,-1)[j,i],np.array(MFPAD_S_phi_LF).reshape(12,6,-1)[6+j,5-i]),np.add(np.array(MFPAD_S_phi_LF).reshape(12,6,-1)[j,i],np.array(MFPAD_S_phi_LF).reshape(12,6,-1)[6+j,5-i]))
            temp_cPECD_std[j,i]=np.divide(np.subtract(cosx_R_std_LF.reshape(12,6,-1)[j,i],cosx_R_std_LF.reshape(12,6,-1)[6+j,5-i]),np.add(cosx_R_std_LF.reshape(12,6,-1)[j,i],cosx_R_std_LF.reshape(12,6,-1)[6+j,5-i]))
            temp_cPECD_err_std[j,i]=fugi.error_calc(cosx_R_std_LF.reshape(12,6,-1)[j,i],cosx_R_std_LF.reshape(12,6,-1)[6+j,5-i])
            temp_cPECD_phi[j,i]=np.divide(np.subtract(cosx_R_phi_LF.reshape(12,6,-1)[j,i],cosx_R_phi_LF.reshape(12,6,-1)[6+j,5-i]),np.add(cosx_R_phi_LF.reshape(12,6,-1)[j,i],cosx_R_phi_LF.reshape(12,6,-1)[6+j,5-i]))
            temp_cPECD_err_phi[j,i]=fugi.error_calc(cosx_R_phi_LF.reshape(12,6,-1)[j,i],cosx_R_phi_LF.reshape(12,6,-1)[6+j,5-i])
            temp_cPECD_opt[j,i]=np.divide(np.subtract(cosx_R_phi_opt_LF.reshape(12,6,-1)[j,i],cosx_R_phi_opt_LF.reshape(12,6,-1)[6+j,5-i]),np.add(cosx_R_phi_opt_LF.reshape(12,6,-1)[j,i],cosx_R_phi_opt_LF.reshape(12,6,-1)[6+j,5-i]))
            temp_cPECD_err_opt[j,i]=fugi.error_calc(cosx_R_phi_opt_LF.reshape(12,6,-1)[j,i],cosx_R_phi_opt_LF.reshape(12,6,-1)[6+j,5-i])

PECD_R=np.array(temp_PECD_R).reshape(72,200,100)
PECD_S=np.array(temp_PECD_S).reshape(72,200,100)
PECD_S_LF=np.array(temp_PECD_S).reshape(72,200,100)

cPECD_std_LF=np.array(temp_cPECD_std).reshape(72,-1)
errPECD_std_LF=np.array(temp_cPECD_err_std).reshape(72,-1)

cPECD_phi_LF=np.array(temp_cPECD_phi).reshape(72,-1)
errPECD_phi_LF=np.array(temp_cPECD_err_phi).reshape(72,-1)

cPECD_phi_opt_LF=np.array(temp_cPECD_opt).reshape(72,-1)
errPECD_phi_LF=np.array(temp_cPECD_err_opt).reshape(72,-1)

#test section

for i, eli in enumerate(MFPAD_S_std_LF):
    for j, elj in enumerate(MFPAD_S_phi_LF):
        if np.all(eli==elj):
            print(i,j)

#test of similarities
#the first and the last one are the same
fail1=[]
fail2=[]
for el1, el2 in zip(MFPAD_S_std_LF,MFPAD_S_phi_LF):
    if np.any(el1!=el2): #np.any is FUNDAMENTAL
        fail1.append(el1)
        fail2.append(el2)

print(np.array(fail1).shape)
print(np.array(fail2).shape)

#too intesive on the CPU
from scipy import interpolate

i2d = interp2d(phi_S_phi_LF[4].reshape(-1),cos_S_phi_LF[4].reshape(-1),MFPAD_S_phi_LF[4].reshape(-1))
znew = i2d(phiM.reshape(-1), cosM.reshape(-1))

#old try to trinagulate
import matplotlib.tri as tri 
triang = tri.Triangulation(phi_S_phi_LF[4].reshape(-1),cos_S_phi_LF[4].reshape(-1))
interpolator = tri.CubicTriInterpolator(triang, MFPAD_S_phi_LF[4].reshape(-1), kind='geom')
phiMM_test, cosMM_test = np.meshgrid(phiM, cosM)
zinterp = interpolator(phiMM_test,cosMM_test)
#zinterp = tri.interp_cubic_geom(phiM,cosM)

#even with 4 subdivided triangles
#refiner = tri.UniformTriRefiner(triang)
#tri_refi, z_test_refi = refiner.refine_field(MFPAD_S_phi_LF[4].reshape(-1), subdiv=1)

#ax[0,1].contourf(phiMM,cosMM,zinterp)
#ax[0,1].set_xlabel('phi [DEG]')
#ax[0,1].set_ylabel('cos(theta) [adm]')
#ax[0,1].set_title("interp2d rot")

#equivalente to interpolator methoh
#to be used after the 3D rotation of the MFPAD passing through cartesian coordiantes
from scipy.interpolate import griddata
zi = griddata((phi_S_phi_LF[4].reshape(-1),cos_S_phi_LF[4].reshape(-1)), MFPAD_S_phi_LF[4].reshape(-1), (phiMM, cosMM), method='linear')
zi_zero = np.nan_to_num(zi,np.nan)

#NOTE: after the triangluation one has to use phiMM and cosMM
fig1, ax = plt.subplots(1,3, figsize=(15,5), constrained_layout=True)
#fig1, ax = plt.subplots(constrained_layout=True)

#ax[0].contourf(triang,zinterp.reshape(-1))
#ax[0].contourf(phiMM,cosMM,MFPAD_S_phi_LF[4].T)
ax[0].contourf(phiMM,cosMM,MFPAD_S_phi[71].T)
#ax[0].contourf(phiMM,cosMM,MFPAD_S_phi_LF_norm_sum.T)
#ax[0].contourf(phiMM, cosMM, MFPAD_temp)
#ax[0].contourf(znew.T)
ax[0].set_xlabel('\u03C6 [DEG]')
ax[0].set_ylabel('cos\u03D1 [adm]')
ax[0].set_title("original")

ax[1].contourf(phiM,cosM,MFPAD_S_phi_LF[71].T)
ax[1].set_xlabel('\u03C6 [DEG]')
ax[1].set_ylabel('cos\u03D1 [adm]')
ax[1].set_title("rot3D with cartesian")

cs, ax[2]=fugi.plot_interpolation(phiM,cosM,MFPAD_S_phi_LF[71].T,ax=ax[2],xstep=1,ystep=0.1)
ax[2].set_xlabel('\u03C6 [DEG]')
ax[2].set_ylabel('cos\u03D1 [adm]')
ax[2].set_title("rot3D with cartesian")

#fig1.savefig('../PYTHON_graphs/OUTPUTS/Till_rotaton.png', figsize=(14,5), dpi=144, constrained_layout=True, transparent=False)

In [ ]:
#fugi = reload(fugi)
fig, axes = plt.subplots(6,12, figsize=(25, 12), dpi=40, sharex='col', sharey='row')
#NOTE for cos ordered the scannig order of axis has to be np.flip(axes.T,axis=1).ravel()
for i,ax in enumerate(np.flip(axes.T,axis=1).ravel()):
#for i,ax in enumerate(axes.ravel()):
    cs, ax=fugi.plot_interpolation(phiM,cosM,(PECD_R[i]*100).T,ax=ax,ystep=0.01, cmap=cmap_temp, limits=[-80.,80.])
    ax.tick_params(labelsize=12)
    
    if i == 4:
        ax.spines[:].set_color('blue')
        ax.spines[:].set_linewidth(5)
    elif i == 37:
        ax.spines[:].set_color('red')
        ax.spines[:].set_linewidth(5)

cbar = fig.colorbar(cs, ax=axes.ravel().tolist(), anchor=(1.5,1), pad=-3.5)
cbar.ax.set_ylabel('PECD %')
fugi.overlaygraph(fig)
fig.savefig("../PYTHON_graphs/OUTPUTS/Theory/72PECD_R_phi_"+en+".png", dpi=300, transparent=False, bbox_inches='tight')

if noprint:
    plt.close()
    #plt.show()
else:
    plt.show()

In [ ]:
#fugi = reload(fugi)
fig, axes = plt.subplots(6,12, figsize=(25, 12), dpi=40, sharex='col', sharey='row')
#NOTE for cos ordered the scannig order of axis has to be np.flip(axes.T,axis=1).ravel()
for i,ax in enumerate(np.flip(axes.T,axis=1).ravel()):
#for i,ax in enumerate(axes.ravel()):
    cs, ax=fugi.plot_interpolation(phiM,cosM,(PECD_S[i]*100).T,ax=ax,ystep=0.01, cmap=cmap_temp, limits=[-80.,80.])
    ax.tick_params(labelsize=12)
    
    if i == 4:
        ax.spines[:].set_color('blue')
        ax.spines[:].set_linewidth(5)
    elif i == 37:
        ax.spines[:].set_color('red')
        ax.spines[:].set_linewidth(5)

cbar = fig.colorbar(cs, ax=axes.ravel().tolist(), anchor=(1.5,1), pad=-3.5)
cbar.ax.set_ylabel('PECD %')
fugi.overlaygraph(fig)
fig.savefig("../PYTHON_graphs/OUTPUTS/Theory/72PECD_S_phi_"+en+".png", dpi=300, transparent=False, bbox_inches='tight')

if noprint:
    #plt.close()
    plt.show()
else:
    plt.show()

In [ ]:
fig,ax = plt.subplots(2,1, figsize=(10, 18), sharex=True)
aspect=(phiM.max()/cosM.max())

cs,ax[0]=fugi.plot_interpolation(phiM,cosM,(PECD_S[0]*100).T, ax=ax[0], cmap=cmap_temp, limits=[-80.,80.], kind="linear", xstep=5, ystep=0.05, n=15, gaussian=1.2)
ax[0].set_ylabel('cos\u03D1 [adm]')
#ax[0].set_xlabel('\u03C6 [DEG]')
ax[0].set_aspect(aspect)

cs,ax[1]=fugi.plot_interpolation(phiM,cosM,(PECD_S[4]*100).T, ax=ax[1], cmap=cmap_temp, limits=[-80.,80.], kind="linear", xstep=5, ystep=0.05, n=15, gaussian=1.2)
ax[1].set_ylabel('cos\u03D1 [adm]')
ax[1].set_xlabel('\u03C6 [DEG]')
ax[1].set_aspect(aspect)

cbar = fig.colorbar(cs, ax=ax.ravel().tolist(), location="top", orientation="horizontal", shrink=0.6)
cbar.ax.locator_params(tight=True, nbins=8)
cbar.ax.set_xlabel('PECD %', ma="right")

fig.savefig("../PYTHON_graphs/OUTPUTS/Theory/0-4PECD_"+en+"_S_phi_exp.png", dpi=300, transparent=False, bbox_inches='tight')

if noprint:
    #plt.close()
    plt.show()
else:
    plt.show()

In [ ]:
fig,ax = plt.subplots(2,1, figsize=(10, 18), sharex=True)
aspect=(phiM.max()/cosM.max())

cs,ax[0]=fugi.plot_interpolation(phiM,cosM,(PECD_R[0]*100).T, ax=ax[0], cmap=cmap_temp, limits=[-80.,80.], kind="linear", xstep=5, ystep=0.05, n=15, gaussian=1.2)
ax[0].set_ylabel('cos\u03D1 [adm]')
#ax[0].set_xlabel('\u03C6 [DEG]')
ax[0].set_aspect(aspect)

cs,ax[1]=fugi.plot_interpolation(phiM,cosM,(PECD_R[4]*100).T, ax=ax[1], cmap=cmap_temp, limits=[-80.,80.], kind="linear", xstep=5, ystep=0.05, n=15, gaussian=1.2)
ax[1].set_ylabel('cos\u03D1 [adm]')
ax[1].set_xlabel('\u03C6 [DEG]')
ax[1].set_aspect(aspect)

cbar = fig.colorbar(cs, ax=ax.ravel().tolist(), location="top", orientation="horizontal", shrink=0.6)
cbar.ax.locator_params(tight=True, nbins=8)
cbar.ax.set_xlabel('PECD %', ma="right")

fig.savefig("../PYTHON_graphs/OUTPUTS/Theory/0-4PECD_"+en+"_R_phi_exp.png", dpi=300, transparent=False, bbox_inches='tight')

if noprint:
    #plt.close()
    plt.show()
else:
    plt.show()

In [ ]:
fig, axes = plt.subplots(6,12, figsize=(25, 12), dpi=40, sharex='col', sharey='row')
#NOTE for cos ordered the scannig order of axis has to be np.flip(axes.T,axis=1).ravel()
for i,ax in enumerate(np.flip(axes.T,axis=1).ravel()):
#for i,ax in enumerate(axes.ravel()):

    cs, ax=fugi.plot_interpolation(phiM,cosM,(PECD_S_LF[i]*100).T,ax=ax,ystep=0.01, cmap=cmap_temp, limits=[-80.,80.])
    ax.tick_params(labelsize=12)
    if i == 4:
        ax.spines[:].set_color('blue')
        ax.spines[:].set_linewidth(5)
    elif i == 37:
        ax.spines[:].set_color('red')
        ax.spines[:].set_linewidth(5)

cbar = fig.colorbar(cs, ax=axes.ravel().tolist(), anchor=(1.5,1), pad=-2.5)
cbar.ax.set_ylabel('PECD %')
fugi.overlaygraph(fig)
fig.savefig("../PYTHON_graphs/OUTPUTS/Theory/72PECD_S_phi_LF_"+en+".png", dpi=300, transparent=False, bbox_inches='tight')

if noprint:
    plt.close()
    #plt.show()
else:
    plt.show()

#fugi = reload(fugi)
if MFPAD == True:
    #all the 72 plots S ENANTIOMER MOLECULAR FRAME
    #all the 72 plots S ENANTIOMER LAB FRAME: useless, the comparison is in the LF
    #the theoretical MFPAD_std has the form 1_1 ... 1_12, 2_1 ... 2_12 therefore uses axes.ravel()
    #the theoretical MFPAD_phi has the form 6_1 ... 1_1, 6_12 ... 1_12 therefore uses np.flip(axes.T,axis=1).ravel()
    
    #fig2a, axes = plt.subplots(6,12, figsize=(50, 24), sharex='col', sharey='row')
    fig2a, axes = plt.subplots(6,12, figsize=(25, 12), dpi=40, sharex='col', sharey='row')

    #NOTE for cos ordered the scannig order of axis has to be np.flip(axes.T,axis=1).ravel()
    for i,ax in enumerate(np.flip(axes.T,axis=1).ravel()):
        cs, ax=fugi.plot_interpolation(phiM,cosM,MFPAD_S_phi[i].T,ax=ax,ystep=0.01)
        ax.tick_params(labelsize=12)
        ax.text(0, 0, "S_en "+str(cosphi_th_phi_cflip[i]), ha="center", va="center", color="red",fontsize=18)
    
    fugi.overlaycbar(fig,cs,axes,MFPAD=True)
    fugi.overlaygraph(fig2a)
    #fig2a.savefig("../PYTHON_graphs/OUTPUTS/Theory/72MFPAD_S_phi_"+en+".png", dpi=300, transparent=False, bbox_inches='tight')
    if noprint:
        plt.close()
    else:
        plt.show()

In [ ]:
#fugi = reload(fugi)
if MFPAD == True:
    #all the 72 plots S ENANTIOMER MOLECULAR FRAME
    #the theoretical MFPAD_std has the form 1_1 ... 1_12, 2_1 ... 2_12 therefore uses axes.ravel()
    #the theoretical MFPAD_phi has the form 6_1 ... 1_1, 6_12 ... 1_12 therefore uses np.flip(axes.T,axis=1).ravel()

    #fig, axes = plt.subplots(6,12, figsize=(50, 24), sharex='col', sharey='row')
    fig, axes = plt.subplots(6,12, figsize=(25, 12), dpi=40, sharex='col', sharey='row')

    #NOTE for cos ordered the scannig order of axis has to be np.flip(axes.T,axis=1).ravel()
    for i,ax in enumerate(axes.ravel()):
        cs, ax=fugi.plot_interpolation(phiM,cosM,MFPAD_S_std[i].T,ax=ax,ystep=0.01)
        ax.tick_params(labelsize=12)
        #ax.text(0, 0, "S_en "+str(cosphi_th_std[i]), ha="center", va="center", color="red",fontsize=18)
    
    fugi.overlaycbar(fig,cs,axes,MFPAD=True)
    fugi.overlaygraph(fig)
    fig.savefig("../PYTHON_graphs/OUTPUTS/Theory/72MFPAD_S_original_"+en+".png", dpi=300, transparent=False, bbox_inches='tight')
    
    if noprint:
        plt.close()
    else:
        plt.show()

In [ ]:
#fugi = reload(fugi)
if MFPAD == True:
    #all the 72 plots S ENANTIOMER MOLECULAR FRAME
    #the theoretical MFPAD_std has the form 1_1 ... 1_12, 2_1 ... 2_12 therefore uses axes.ravel()
    #the theoretical MFPAD_phi has the form 6_1 ... 1_1, 6_12 ... 1_12 therefore uses np.flip(axes.T,axis=1).ravel()

    #fig, axes = plt.subplots(6,12, figsize=(50, 24), sharex='col', sharey='row')
    fig, axes = plt.subplots(6,12, figsize=(25, 12), dpi=40, sharex='col', sharey='row')

    #NOTE for cos ordered the scannig order of axis has to be np.flip(axes.T,axis=1).ravel()
    for i,ax in enumerate(axes.ravel()):
        cs, ax=fugi.plot_interpolation(phiM,cosM,MFPAD_S_phi[i].T,ax=ax,ystep=0.01, cmap=cmap_temp)
        ax.tick_params(labelsize=12)
        #ax.text(0, 0, "S_en "+str(cosphi_th_std[i]), ha="center", va="center", color="red",fontsize=18)

    fugi.overlaycbar(fig,cs,axes,MFPAD=True)
    fugi.overlaygraph(fig)
    fig.savefig("../PYTHON_graphs/OUTPUTS/Theory/72MFPAD_S_original_cmap_"+en+".png", dpi=300, transparent=False, bbox_inches='tight')
    
    if noprint:
        plt.close()
    else:
        plt.show()

In [ ]:
#this can be all added togheter to create the total sum
#fig,ax = plt.subplots(2,1, figsize=(8, 12), sharex='col', tight_layout=True)
fig,ax = plt.subplots(2,1, figsize=(10, 18), sharex=True)
aspect=(phiM.max()/cosM.max())

cs,ax[0]=fugi.plot_interpolation(phiM,cosM,MFPAD_S_phi[37].T, ax=ax[0], ystep=0.01, gaussian=7.)
contrast=cs.get_array().max()/cs.get_array().min()
ax[0].set_ylabel('cos\u03D1 [adm]')
#ax[0].set_xlabel('\u03C6 [DEG]')
ax[0].set_aspect(aspect)

cs,ax[1]=fugi.plot_interpolation(phiM,cosM,MFPAD_S_phi[4].T, ax=ax[1], ystep=0.01, gaussian=7.)
ax[1].set_ylabel('cos\u03D1 [adm]')
ax[1].set_xlabel('\u03C6 [DEG]')
ax[1].set_aspect(aspect)

cbar = fig.colorbar(cs, ax=ax.ravel().tolist(), location="top", orientation="horizontal", shrink=0.6)
cbar.set_ticks([cs.get_array().min(),cs.get_array().max()])
cbar.set_ticklabels([1,f"{contrast:.2f}"])
cbar.ax.set_xlabel('contrast', rotation="horizontal", rotation_mode="anchor", loc="center")
#cbar.ax.set_xlabel('normalised counts', rotation="horizontal", rotation_mode="anchor", loc="center")

fig.savefig("../PYTHON_graphs/OUTPUTS/Theory/37-4MFPAD_S_phi_"+CH+"_LF_"+en+".png", dpi=300, transparent=False, bbox_inches='tight')

if noprint:
    #plt.close()
    plt.show()
else:
    plt.show()

In [ ]:
#EXPORT FOR 3D RENDERING
%store MFPAD_S_phi[0]
%store MFPAD_S_phi[4]

#fugi = reload(fugi)
if MFPAD == True:
    #all the 72 plots S ENANTIOMER MOLECULAR FRAME
    #the theoretical MFPAD_std has the form 1_1 ... 1_12, 2_1 ... 2_12 therefore uses axes.ravel()
    #the theoretical MFPAD_cos has the form 6_1 ... 6_12, 5_1 ... 5_12 therefore uses np.flip(axes.T,axis=1).ravel()
    
    #fig2a, axes = plt.subplots(6,12, figsize=(50, 24), sharex='col', sharey='row')
    fig2a, axes = plt.subplots(6,12, figsize=(25, 12), dpi=40, sharex='col', sharey='row')

    #NOTE for cos ordered the scannig order of axis has to be np.flip(axes.T,axis=1).ravel()
    for i,ax in enumerate(np.flip(axes.T,axis=1).ravel()):
        cs, ax=fugi.plot_interpolation(phiM,cosM,MFPAD_S_phi[i].T,ax=ax,ystep=0.01)
        ax.tick_params(labelsize=12)
        #ax.text(0, 0, "S_en "+str(cosphi_th_phi_cflip[i]), ha="center", va="center", color="red",fontsize=18)
    
    fugi.overlaycbar(fig,cs,axes,MFPAD=True)
    fugi.overlaygraph(fig2a)
    #fig2a.savefig("../PYTHON_graphs/OUTPUTS/Theory/72MFPAD_S_phi_"+en+".png", dpi=300, transparent=False, bbox_inches='tight')
    if noprint:
        plt.close()
    else:
        plt.show()

if MFPAD == True:
    #all the 72 plots S ENANTIOMER LAB FRAME: useless, the comparison is in the LF
    #the theoretical MFPAD_std has the form 1_1 ... 1_12, 2_1 ... 2_12 therefore uses axes.ravel()
    #the theoretical MFPAD_cos has the form 6_1 ... 6_12, 5_1 ... 5_12 therefore uses np.flip(axes.T,axis=1).ravel()
    
    #fig2a, axes = plt.subplots(6,12, figsize=(50, 24), sharex='col', sharey='row')
    fig2a, axes = plt.subplots(6,12, figsize=(25, 12), dpi=40, sharex='col', sharey='row')

    #NOTE for cos ordered the scannig order of axis has to be np.flip(axes.T,axis=1).ravel()
    for i,ax in enumerate(np.flip(axes.T,axis=1).ravel()):
        cs, ax=fugi.plot_interpolation(phiM,cosM,MFPAD_S_phi[i].T,ax=ax,ystep=0.01,cmap=cmap_temp)
        ax.tick_params(labelsize=12)
        #ax.text(0, 0, str(cosphi_th_phi_cflip[i]), ha="center", va="center", color="red",fontsize=12)
    
    fugi.overlaycbar(fig,cs,axes,MFPAD=True)
    fugi.overlaygraph(fig2a)
    #fig2a.savefig("../PYTHON_graphs/OUTPUTS/Theory/72MFPAD_S_phi_cmap_"+en+".png", dpi=300, transparent=False, bbox_inches='tight')
    if noprint:
        plt.close()
    else:
        plt.show()

#fugi = reload(fugi)
if MFPAD == True and original == True:
    #all the 72 plots S ENANTIOMER LAB FRAME: useless, the comparison is in the LF
    #the theoretical MFPAD_std has the form 1_1 ... 1_12, 2_1 ... 2_12 therefore uses axes.ravel()
    #the theoretical MFPAD_cos has the form 6_1 ... 6_12, 5_1 ... 5_12 therefore uses np.flip(axes.T,axis=1).ravel()

    #fig2a, axes = plt.subplots(6,12, figsize=(50, 24), sharex='col', sharey='row')
    fig2a, axes = plt.subplots(6,12, figsize=(25, 12), dpi=40, sharex='col', sharey='row')

   #NOTE for cos ordered the scannig order of axis has to be np.flip(axes.T,axis=1).ravel()
    for i,ax in enumerate(axes.ravel()):
        cs, ax=fugi.plot_interpolation(phiM,cosM,MFPAD_S_std_LF[i].T,ax=ax,ystep=0.01,cmap=cmap_temp)
        ax.tick_params(labelsize=12)
        ax.text(0, 0, "S_en "+str(cosphi_th_std[i]), ha="center", va="center", color="red",fontsize=18)

    fugi.overlaycbar(fig,cs,axes,MFPAD=True)
    fugi.overlaygraph(fig2a)
    #fig2a.savefig("../PYTHON_graphs/OUTPUTS/Theory/72MFPAD_S_original_LF_cmap_"+en+".png", dpi=300, transparent=False, bbox_inches='tight')
    if noprint:
        plt.close()
    else:
        plt.show()

#fugi = reload(fugi)
if MFPAD == True and original == True:
    #all the 72 plots S ENANTIOMER LAB FRAME: useless, the comparison is in the LF
    #the theoretical MFPAD_std has the form 1_1 ... 1_12, 2_1 ... 2_12 therefore uses axes.ravel()
    #the theoretical MFPAD_cos has the form 6_1 ... 6_12, 5_1 ... 5_12 therefore uses np.flip(axes.T,axis=1).ravel()

    #fig2a, axes = plt.subplots(6,12, figsize=(50, 24), sharex='col', sharey='row')
    fig2a, axes = plt.subplots(6,12, figsize=(25, 12), dpi=40, sharex='col', sharey='row')

   #NOTE for cos ordered the scannig order of axis has to be np.flip(axes.T,axis=1).ravel()
    #for i,ax in enumerate(axes.ravel()):
    for i,ax in enumerate(np.flip(axes.T,axis=1).ravel()):

        cs, ax=fugi.plot_interpolation(phiM,cosM,MFPAD_S_phi_LF[i].T,ax=ax,ystep=0.01,cmap=cmap_temp)
        ax.tick_params(labelsize=12)
        ax.text(0, 0, "S_en "+str(cosphi_th_phi[i]), ha="center", va="center", color="red",fontsize=18)

    fugi.overlaycbar(fig,cs,axes,MFPAD=True)
    fugi.overlaygraph(fig2a)
    #fig2a.savefig("../PYTHON_graphs/OUTPUTS/Theory/72MFPAD_S_phi_LF_cmap_"+en+".png", dpi=300, transparent=False, bbox_inches='tight')
    if noprint:
        plt.close()
    else:
        plt.show()

In [ ]:
#fugi = reload(fugi)
if MFPAD == True:
    #fig, axes = plt.subplots(6,12, figsize=(50, 24), sharex='col', sharey='row')
    fig, axes = plt.subplots(6,12, figsize=(25, 12), dpi=40, sharex='col', sharey='row')

   #NOTE for cos ordered the scannig order of axis has to be np.flip(axes.T,axis=1).ravel()
    #for i,ax in enumerate(axes.ravel()):
    for i,ax in enumerate(np.flip(axes.T,axis=1).ravel()):

        cs, ax=fugi.plot_interpolation(phiM,cosM,MFPAD_S_phi_rot[i].T,ax=ax,ystep=0.01,gaussian=7.)
        ax.tick_params(labelsize=12)
        #ax.text(0, 0, "S_en "+str(cosphi_th_phi[i]), ha="center", va="center", color="red",fontsize=18)

    fugi.overlaycbar(fig,cs,axes,MFPAD=True)
    fugi.overlaygraph(fig)
    fig.savefig("../PYTHON_graphs/OUTPUTS/Theory/72MFPAD_S_opt_"+CH+"_LF_"+en+".png", dpi=300, transparent=False, bbox_inches='tight')
    if noprint:
        plt.close()
    else:
        plt.show()

In [ ]:
#this can be all added togheter to create the total sum
#fig,ax = plt.subplots(2,1, figsize=(8, 12), sharex='col', tight_layout=True)
fig,ax = plt.subplots(2,1, figsize=(10, 18), sharex=True)
aspect=(phiM.max()/cosM.max())

cs,ax[0]=fugi.plot_interpolation(phiM,cosM,MFPAD_S_phi_rot[37].T, ax=ax[0], ystep=0.01, gaussian=7.)
contrast=cs.get_array().max()/cs.get_array().min()
ax[0].set_ylabel('cos\u03D1 [adm]')
#ax[0].set_xlabel('\u03C6 [DEG]')
ax[0].set_aspect(aspect)

cs,ax[1]=fugi.plot_interpolation(phiM,cosM,MFPAD_S_phi_rot[4].T, ax=ax[1], ystep=0.01, gaussian=7.)
ax[1].set_ylabel('cos\u03D1 [adm]')
ax[1].set_xlabel('\u03C6 [DEG]')
ax[1].set_aspect(aspect)

cbar = fig.colorbar(cs, ax=ax.ravel().tolist(), location="top", orientation="horizontal", shrink=0.6)
cbar.set_ticks([cs.get_array().min(),cs.get_array().max()])
cbar.set_ticklabels([1,f"{contrast:.2f}"])
cbar.ax.set_xlabel('contrast', rotation="horizontal", rotation_mode="anchor", loc="center")
#cbar.ax.set_xlabel('normalised counts', rotation="horizontal", rotation_mode="anchor", loc="center")

fig.savefig("../PYTHON_graphs/OUTPUTS/Theory/37-4MFPAD_S_opt_"+CH+"_LF_"+en+".png", dpi=300, transparent=False, bbox_inches='tight')

if noprint:
    #plt.close()
    plt.show()
else:
    plt.show()

In [ ]:
if MFPAD == True:
    #all the 72 plots S ENANTIOMER LAB FRAME: useless, the comparison is in the LF
    #the theoretical MFPAD_std has the form 1_1 ... 1_12, 2_1 ... 2_12 therefore uses axes.ravel()
    #the theoretical MFPAD_cos has the form 6_1 ... 6_12, 5_1 ... 5_12 therefore uses np.flip(axes.T,axis=1).ravel()
    
    #fig, axes = plt.subplots(6,12, figsize=(50, 24), sharex='col', sharey='row')
    fig, axes = plt.subplots(6,12, figsize=(25, 12), dpi=40, sharex='col', sharey='row')

    #NOTE for cos ordered the scannig order of axis has to be np.flip(axes.T,axis=1).ravel()
    for i,ax in enumerate(np.flip(axes.T,axis=1).ravel()):
        cs, ax=fugi.plot_interpolation(phiM,cosM,MFPAD_R_phi[i].T,ax=ax,ystep=0.01,cmap=cmap_temp)
        ax.tick_params(labelsize=12)
        #ax.text(0, 0, str(cosphi_th_phi_cflip[i]), ha="center", va="center", color="red",fontsize=12)
    
    fugi.overlaycbar(fig,cs,axes,MFPAD=True)
    fugi.overlaygraph(fig)
    fig.savefig("../PYTHON_graphs/OUTPUTS/Theory/72MFPAD_R_phi_cmap_"+en+".png", dpi=300, transparent=False, bbox_inches='tight')
    if noprint:
        plt.close()
    else:
        plt.show()

In [ ]:
if MFPAD == True:
    #all the 72 plots S ENANTIOMER LAB FRAME: useless, the comparison is in the LF
    #the theoretical MFPAD_std has the form 1_1 ... 1_12, 2_1 ... 2_12 therefore uses axes.ravel()
    #the theoretical MFPAD_cos has the form 6_1 ... 6_12, 5_1 ... 5_12 therefore uses np.flip(axes.T,axis=1).ravel()
    
    #fig, axes = plt.subplots(6,12, figsize=(50, 24), sharex='col', sharey='row')
    fig, axes = plt.subplots(6,12, figsize=(25, 12), dpi=40, sharex='col', sharey='row')

    #NOTE for cos ordered the scannig order of axis has to be np.flip(axes.T,axis=1).ravel()
    for i,ax in enumerate(np.flip(axes.T,axis=1).ravel()):
        cs, ax=fugi.plot_interpolation(phiM,cosM,MFPAD_R_phi[i].T,ax=ax,ystep=0.01)
        ax.tick_params(labelsize=12)
        #ax.text(0, 0, str(cosphi_th_phi_cflip[i]), ha="center", va="center", color="red",fontsize=12)
    
    fugi.overlaycbar(fig,cs,axes,MFPAD=True)
    fugi.overlaygraph(fig)
    fig.savefig("../PYTHON_graphs/OUTPUTS/Theory/72MFPAD_R_phi_"+en+".png", dpi=300, transparent=False, bbox_inches='tight')
    if noprint:
        plt.close()
    else:
        plt.show()

In [ ]:
#PECD rotated
plt.figure(figsize=(3, 3))
plt.plot(errPECD_phi[60],".")
plt.plot(cPECD_phi[60])
plt.plot(cos_projthR_phi[60])
plt.plot(cos_projthS_phi[60])
print(np.sum(cos_projthR_phi[60]))

#PECD for non rotated
plt.figure(figsize=(3, 3))
plt.plot(errPECD_phi_LF[60],".")
plt.plot(cPECD_phi_LF[60])
plt.plot(cosx_R_phi_LF[60])
plt.plot(cosx_S_phi_LF[60])
print(np.sum(cosx_R_phi_LF[60]))

In [ ]:
#comparison of PECD with different cos(theta)
fig1, ax = plt.subplots(1,4, figsize=(10, 2), dpi=40, constrained_layout=True)
#cos along z axis (original)
ax[0].plot(cos_projthR_phi[0],".")
ax[0].plot(cos_projthS_phi[0])

#cos along x axis: [0] is a rotation of -1 in cos(theta) and -180 in phi, i.e. completly symmetric in phi
ax[1].plot(cosx_R_phi_LF[0],".")
ax[1].plot(cosx_S_phi_LF[0])

#phi along y axis (original)
ax[2].plot(phi_projthR_phi[0],".")
ax[2].plot(phi_projthS_phi[0])

#phi along y axis: as for the cos(theta)
ax[3].plot(phi_projthR_phi_LF[0],".")
ax[3].plot(phi_projthS_phi_LF[0])

In [ ]:
#PHI ordered modified
import functions_minuit as fumi
# fumi = reload(fumi)
#param_matrix=np.zeros((6,72)) #alternative nparray initialization
param_matrix_phi = [[0 for i in range(6)] for j in range(72)] #i col, j row

#fig, axes = plt.subplots(6,12, figsize=(50, 24), constrained_layout=True, sharex=True , sharey=True)
fig, axes = plt.subplots(6,12, figsize=(25, 12), constrained_layout=True, sharex=True , sharey=True)
custom_xlim = (-1, 1)
custom_ylim = (-0.6,0.6) #custom_ylim = (-1,1)

#limits=[(-1, 1), (-1, 2), (-1, 1), (-1, 1), (-1, 1), (-1, 1)]
limits=[(-1, 1), (-1, 2), (None, None), (None, None), (None, None), (None, None)]

for i,ax in enumerate(np.flip(axes.T,axis=1).ravel()):
    #NOTE: x_data has to be monotonically INCREASING!
    x_data = cosM;y_data = cPECD_phi_LF[i];
    #original in MF
    #x_data = np.flip(cosM);y_data = cPECD_cos[i];#y_err = errPECD_cos[i] 

    m=fumi.iminuit_noerr(x_data,y_data,limits)
    fumi.plotminuit_noerr(x_data,y_data,m,ax,custom_xlim,custom_ylim)

    for counter,el in enumerate(m.parameters): #len = 6
        param_matrix_phi[i][counter]=((m.values[el], m.errors[el]))

param_matrix_phi=np.array(param_matrix_phi)

fig.supylabel("PECD", x=0.91, y=0.82)
fugi.overlaygraph(fig,wspace=0.08, hspace=0.08)

fig.savefig("../PYTHON_graphs/OUTPUTS/Theory/b1matrix_R-S_LF_phi_theory_"+en+".png", dpi=300, transparent=False, bbox_inches='tight')

if noprint:
    plt.show()
    #plt.close()
else:
    plt.show()

fugi = reload(fugi)
left without imits on purpose
fig=fugi.plotgo_single(param_matrix_phi,xgo_phi,ygo_phi,name="b1map_phi_LF_"+en+"_theory")
fig.show()

In [ ]:
#fugi = reload(fugi)
fig,ax= plt.subplots(1, figsize=(12, 10))
#cs,ax=fugi.plot_interpolation(xgo_uniq,ygo_uniq,param_matrix_phi[:,0,0].reshape(len(xgo_uniq),-1).T*100, ax=ax, cmap=cmap_temp)
cs,ax=fugi.plot_interpolation(np.array(xgo_phi).reshape(12,6),np.array(ygo_phi).reshape(12,6),param_matrix_phi[:,0,0].reshape(12,6)*100, ax=ax, cmap=cmap_temp)

ax.set_title("b1 map "+en)
#aspect=(xgo_uniq.max()/ygo_uniq.max())
ax.set_xlabel('\u03C6$_{ph}$ [DEG]')
ax.set_ylabel('cos\u03D1$_{ph}$ [adm]')
#ax.set_aspect(aspect)

#cbar = fugi.clippedcolorbar(cs,shrink=0.9, ax=cs)
cbar = fig.colorbar(cs,shrink=0.9, ax=ax)
#cbar.set_ticks([totMFPAD.min(),totMFPAD.max()])
cbar.ax.set_ylabel('% of asymmetry')

fig.savefig("../PYTHON_graphs/OUTPUTS/Theory/b1map_phi_LF_"+en+"_theory.png", dpi=300, transparent=False, bbox_inches='tight')

In [ ]:
#fugi = reload(fugi)
fig,ax= plt.subplots(1, figsize=(12, 10))
cs = ax.pcolormesh(np.array(xgo_phi).reshape(12,6),np.array(ygo_phi).reshape(12,6),param_matrix_phi[:,0,0].reshape(12,6)*100,cmap=cmap_temp)

ax.set_title("b1 map "+en)
aspect=(xgo_uniq.max()/ygo_uniq.max())
ax.set_xlabel('\u03C6$_{ph}$ [DEG]')
ax.set_xticks(xgo_uniq); ax.xaxis.set_tick_params(rotation=-45)
ax.set_ylabel('cos\u03D1$_{ph}$ [adm]')
ax.set_yticks(ygo_uniq)
ax.set_aspect(aspect)

cbar = fig.colorbar(cs,shrink=0.9, ax=ax)
cbar.ax.set_ylabel('% of asymmetry')

fig.savefig("../PYTHON_graphs/OUTPUTS/Theory/b1map_phi_LF_RAW_"+en+"_theory.png", dpi=300, transparent=False, bbox_inches='tight')

#reload(fugi)
fig,ax= plt.subplots(1, figsize=(12, 10))
cs=ax.tricontourf(xgo_phi,ygo_phi,fugi.smoothgauss(param_matrix_phi[:,0,0],0.3,0.6), cmap=cmap_temp, extend="both", levels=20)
ax.set_title("b1 map "+en)

aspect=(phiM.max()/cosM.max())
ax.set_xlabel('phi [DEG]')
ax.set_ylabel('cos(theta) [adm]')
cbar = fugi.clippedcolorbar(cs,shrink=0.9, ax=ax)
#cbar.set_ticks([totMFPAD.min(),totMFPAD.max()])
cbar.ax.set_ylabel('% of asymmetry')
ax.set_aspect(aspect)

fig.savefig("../PYTHON_graphs/OUTPUTS/Theory/b1map_phi_LF_"+en+"_theory.png", figsize=(50, 24), dpi=144, transparent=False)

left without limits on purpose
fig=fugi.plotgo_multiple(param_matrix_phi,xgo_phi,ygo_phi,name="b1-6map_phi_LF_"+en+"_theory")
fig.show()

fugi = reload(fugi)
limits=[[-1,2,0.25],[-0.18,0.18,0.025]]
fig=fugi.plotgo_multiple(param_matrix_cos,xgo_cos,ygo_cos,name="b1-6map_S_CR-CL_cos_"+CH+"_"+en+"_exp",limits=limits)
fig.show()

fig,axes= plt.subplots(6,1, figsize=(10, 40), sharex=True)
cs=ax.tricontourf(xgo_cos,ygo_cos,fugi.smoothgauss(param_matrix_cos[:,0,0],0.3,0.6),cmap=cmap_temp,levels=20)
cs=ax.tricontourf(xgo_cos,ygo_cos,fugi.smoothgauss(param_matrix_cos[:,0,0],0.3,0.6), vmin=-0.2, vmax=0.2,cmap=cmap_temp,levels=20)
cs=ax.pcolormesh(np.array(xgo_cos),np.array(ygo_cos),fugi.smoothgauss(param_matrix_cos[:,0,0],0.3,0.6),cmap=cmap_temp)

In [ ]:
#fugi = reload(fugi)
fig,axes= plt.subplots(6,1, figsize=(10, 40), sharex=True)
fig.suptitle("b1 map "+en)

for ax,i in zip(axes,range(6)):
    #cs,ax=fugi.plot_interpolation(xgo_uniq,ygo_uniq,param_matrix_phi[:,i,0].reshape(len(xgo_uniq),-1).T, ax=ax, cmap=cmap_temp)
    cs,ax=fugi.plot_interpolation(np.array(xgo_phi).reshape(12,6),np.array(ygo_phi).reshape(12,6),param_matrix_phi[:,i,0].reshape(12,6), ax=ax, cmap=cmap_temp)
    #cs,ax=fugi.plot_interpolation(xgo_cos,ygo_cos,param_matrix_cos[:,0,0].T*100, ax=ax, cmap=cmap_temp)

    aspect=(xgo_uniq.max()/ygo_uniq.max())
    ax.set_ylabel('cos\u03D1$_{ph}$ [adm]')
    ax.set_aspect(aspect)
    cbar = fig.colorbar(cs,shrink=0.9, ax=ax)
    cbar.ax.set_ylabel("asymmetry")
    if i==5:
        ax.set_xlabel('\u03C6$_{ph}$ [DEG]')
fig.tight_layout()

fig.savefig("../PYTHON_graphs/OUTPUTS/Theory/b1-6map_phi_LF_"+en+"_theory.png", dpi=300, transparent=False, bbox_inches='tight')

if noprint:
    plt.show()
    #plt.close()
else:
    plt.show()

In [ ]:
#fumi = reload(fumi)
if std_calculations == True:
    #param_matrix=np.zeros((6,72)) #alternative nparray initialization
    param_matrix_std = [[0 for i in range(6)] for j in range(72)] #i col, j row

    #fig, axes = plt.subplots(6,12, figsize=(50, 24), constrained_layout=True, sharex=True , sharey=True)
    fig, axes = plt.subplots(6,12, figsize=(25, 12), constrained_layout=True, sharex=True , sharey=True)
    custom_xlim = (-1, 1)
    custom_ylim = (-0.6,0.6) #custom_ylim = (-1,1)

    #limits=[(-1, 1), (-1, 2), (-1, 1), (-1, 1), (-1, 1), (-1, 1)]
    limits=[(-1, 1), (-1, 2), (None, None), (None, None), (None, None), (None, None)]

    #for i,ax in zip(range(72),axes.ravel()): #fixe because of the photon directions, equal to range(len(cosphi))
    for i,ax in enumerate(np.flip(axes.T,axis=1).ravel()):
        #NOTE: x_data has to be monotonicallY INCREASING!
        x_data = cosM;y_data = cPECD_std_LF[i];
        #x_data = np.flip(cosM);y_data = cPECD_phi_LF[i];y_err = errPECD_LF_cos[i]

        m=fumi.iminuit_noerr(x_data,y_data,limits)
        fumi.plotminuit_noerr(x_data,y_data,m,ax,custom_xlim,custom_ylim)

        for counter,el in enumerate(m.parameters): #len = 6
            param_matrix_std[i][counter]=((m.values[el], m.errors[el])) #6 parameter * 2 becuaseof the error!!

    #FLIPPING BECAUSE OF THE COS SCANNING?
    #param_matrix_std=np.flip(np.array(param_matrix_std).reshape(6,2,72).T,axis=2).reshape(-1) #numpy tranformation!!!
    param_matrix_std=np.array(param_matrix_std)

    fig.supylabel("PECD", x=0.91, y=0.82)
    fugi.overlaygraph(fig,wspace=0.08, hspace=0.08)

    fig.savefig("../PYTHON_graphs/OUTPUTS/Theory/b1matrix_R-S_LF_std_"+en+"_theory.png", dpi=300, transparent=False, bbox_inches='tight')

In [ ]:
#fugi = reload(fugi)
if std_calculations == True:
    fig,ax= plt.subplots(1, figsize=(12, 10))
    
    cs,ax=fugi.plot_interpolation(xgo_flip_uniq,ygo_flip_uniq,np.flip(np.array(param_matrix_std[:,0,0]).reshape(6,12).T,axis=1).reshape(-1)*100, ax=ax,  cmap=cmap_temp)
    #cs=ax.tricontourf(xgo_cos,ygo_cos,fugi.smoothgauss(param_matrix_cos[:,0,0],0.3,0.6), extend="both",cmap=cmap_temp,levels=20)

    ax.set_title("b1 map "+en)
    aspect=(xgo_uniq.max()/ygo_uniq.max())
    ax.set_xlabel('\u03C6$_{ph}$ [DEG]')
    ax.set_ylabel('cos\u03D1$_{ph}$ [adm]')
    ax.set_aspect(aspect)

    #cbar = fugi.clippedcolorbar(cs,shrink=0.9, ax=cs)
    cbar = fig.colorbar(cs,shrink=0.9, ax=ax)
    #cbar.set_ticks([totMFPAD.min(),totMFPAD.max()])
    cbar.ax.set_ylabel('% of asymmetry')

    #fig.savefig("../PYTHON_graphs/OUTPUTS/Theory/b1map_std_LF_"+en+"_theory.png", dpi=300, transparent=False, bbox_inches='tight')

In [ ]:
if std_calculations == True:
    fig,axes= plt.subplots(6,1, figsize=(10, 40), sharex=True)
    #cs=ax.tricontourf(xgo_cos,ygo_cos,fugi.smoothgauss(param_matrix_cos[:,0,0],0.3,0.6),cmap=cmap_temp,levels=20)
    #cs=ax.tricontourf(xgo_cos,ygo_cos,fugi.smoothgauss(param_matrix_cos[:,0,0],0.3,0.6), vmin=-0.2, vmax=0.2,cmap=cmap_temp,levels=20)
    #cs=ax.pcolormesh(np.array(xgo_cos),np.array(ygo_cos),fugi.smoothgauss(param_matrix_cos[:,0,0],0.3,0.6),cmap=cmap_temp)
    fig.suptitle("b1 map "+en)

    for ax,i in zip(axes,range(6)):
        cs,ax=fugi.plot_interpolation(xgo_flip_uniq,ygo_flip_uniq,np.flip(np.array(param_matrix_std[:,i,0]).reshape(6,12).T,axis=1).reshape(-1), ax=ax, cmap=cmap_temp)
        aspect=(xgo_flip_uniq.max()/ygo_flip_uniq.max())
        ax.set_ylabel('cos\u03D1$_{ph}$ [adm]')
        ax.set_aspect(aspect)
        cbar = fig.colorbar(cs,shrink=0.9, ax=ax)
        cbar.ax.set_ylabel("asymmetry")
        if i==5:
            ax.set_xlabel('\u03C6$_{ph}$ [DEG]')
    fig.tight_layout()

    fig.savefig("../PYTHON_graphs/OUTPUTS/Theory/b1-6map_std_LF_"+en+"_theory.png", dpi=300, transparent=False, bbox_inches='tight')

In [ ]:
#PHI ordered modified
#import functions_minuit as fumi
#fumi = reload(fumi)
#param_matrix=np.zeros((6,72)) #alternative nparray initialization
param_matrix_opt = [[0 for i in range(6)] for j in range(72)] #i col, j row

#fig, axes = plt.subplots(6,12, figsize=(50, 24), constrained_layout=True, sharex=True , sharey=True)
fig, axes = plt.subplots(6,12, figsize=(25, 12), constrained_layout=True, sharex=True , sharey=True)
custom_xlim = (-1, 1)
custom_ylim = (-0.6,0.6) #custom_ylim = (-1,1)

#limits=[(-1, 1), (-1, 2), (-1, 1), (-1, 1), (-1, 1), (-1, 1)]
limits=[(-1, 1), (-1, 2), (None, None), (None, None), (None, None), (None, None)]

for i,ax in enumerate(np.flip(axes.T,axis=1).ravel()):
    #NOTE: x_data has to be monotonically INCREASING!
    x_data = cosM;y_data = cPECD_phi_opt_LF[i]
    m=fumi.iminuit_noerr(x_data,y_data,limits)
    fumi.plotminuit_noerr(x_data,y_data,m,ax,custom_xlim,custom_ylim)

    for counter,el in enumerate(m.parameters): #len = 6
        param_matrix_opt[i][counter]=((m.values[el], m.errors[el]))

param_matrix_opt=np.array(param_matrix_opt) #numpy tranformation!!!

fig.supylabel("PECD", x=0.91, y=0.82)
fugi.overlaygraph(fig,wspace=0.08, hspace=0.08)

fig.savefig("../PYTHON_graphs/OUTPUTS/Theory/b1matrix_R-S_LF_phi_opt_"+CH+"_theory_"+en+".png", dpi=300, transparent=False, bbox_inches='tight')

if noprint:
    plt.close()
else:
    plt.show()

In [ ]:
#fugi = reload(fugi)
fig,ax= plt.subplots(1, figsize=(12, 10))
#cs,ax=fugi.plot_interpolation(xgo_uniq,ygo_uniq,param_matrix_opt[:,0,0].reshape(len(xgo_uniq),-1).T*100, ax=ax, cmap=cmap_temp)
cs,ax=fugi.plot_interpolation(np.array(xgo_phi).reshape(12,6),np.array(ygo_phi).reshape(12,6),param_matrix_opt[:,0,0].reshape(12,6)*100, ax=ax, cmap=cmap_temp)

ax.set_title("b1 map "+en)
#aspect=(xgo_uniq.max()/ygo_uniq.max())
ax.set_xlabel('\u03C6$_{ph}$ [DEG]')
ax.set_ylabel('cos\u03D1$_{ph}$ [adm]')
#ax.set_aspect(aspect)

#cbar = fugi.clippedcolorbar(cs,shrink=0.9, ax=cs)
cbar = fig.colorbar(cs,shrink=0.9, ax=ax)
#cbar.set_ticks([totMFPAD.min(),totMFPAD.max()])
cbar.ax.set_ylabel('% of asymmetry')

fig.savefig("../PYTHON_graphs/OUTPUTS/Theory/b1map_phi_opt_"+CH+"_LF_"+en+"_theory.png", dpi=300, transparent=False, bbox_inches='tight')

In [ ]:
fig,axes= plt.subplots(6,1, figsize=(10, 40), sharex=True)
fig.suptitle("b1 map "+en)

for ax,i in zip(axes,range(6)):
    #cs,ax=fugi.plot_interpolation(xgo_uniq,ygo_uniq,param_matrix_opt[:,i,0].reshape(len(xgo_uniq),-1).T, ax=ax, cmap=cmap_temp)
    cs,ax=fugi.plot_interpolation(np.array(xgo_phi).reshape(12,6),np.array(ygo_phi).reshape(12,6),param_matrix_opt[:,i,0].reshape(12,6), ax=ax, cmap=cmap_temp)

    aspect=(xgo_uniq.max()/ygo_uniq.max())
    ax.set_ylabel('cos\u03D1$_{ph}$ [adm]')
    ax.set_aspect(aspect)
    cbar = fig.colorbar(cs,shrink=0.9, ax=ax)
    cbar.ax.set_ylabel("asymmetry")
    if i==5:
        ax.set_xlabel('\u03C6$_{ph}$ [DEG]')
fig.tight_layout()

fig.savefig("../PYTHON_graphs/OUTPUTS/Theory/b1-6map_phi_opt_"+CH+"_LF_"+en+"_theory.png", dpi=300, transparent=False, bbox_inches='tight')

if noprint:
    plt.close()
else:
    plt.show()

In [ ]:
#before rotation, in the MF
#cos ordered modified with error

#param_matrix=np.zeros((6,72)) #alternative nparray initialization
param_matrix_MF = [[0 for i in range(6)] for j in range(72)] #i col, j row

#fig, axes = plt.subplots(6,12, figsize=(50, 24), constrained_layout=True, sharex=True , sharey=True)
fig, axes = plt.subplots(6,12, figsize=(25, 12), constrained_layout=True, sharex=True , sharey=True)
custom_xlim = (-1, 1)
custom_ylim = (-0.6,0.6) #custom_ylim = (-1,1)

#limits=[(-1, 1), (-1, 2), (-1, 1), (-1, 1), (-1, 1), (-1, 1)]
limits=[(-1, 1), (-1, 2), (None, None), (None, None), (None, None), (None, None)]

#for i,ax in zip(range(72),axes.ravel()): #fixe because of the photon directions, equal to range(len(cosphi))
for i,ax in enumerate(np.flip(axes.T,axis=1).ravel()):
    #NOTE: x_data has to be monotonicalli INCREASING!
    #x_data = np.flip(cosM);y_data = cPECD_phi[i];y_err = errPECD_LF_cos[i]
    #original in MF
    x_data = cosM;y_data = cPECD_phi[i];

    m=fumi.iminuit_noerr(x_data,y_data,limits)
    fumi.plotminuit_noerr(x_data,y_data,m,ax,custom_xlim,custom_ylim)

    for counter,el in enumerate(m.parameters): #len = 6
        param_matrix_MF[i][counter]=((m.values[el], m.errors[el]))

param_matrix_MF=np.array(param_matrix_MF) #numpy tranformation!!!

fig.supylabel("PECD", x=0.91, y=0.82)
fugi.overlaygraph(fig,wspace=0.08, hspace=0.08)

fig.savefig("../PYTHON_graphs/OUTPUTS/Theory/b1matrix_R-S_MF_phi_"+en+"_theory.png", dpi=300, transparent=False, bbox_inches='tight')

if noprint:
    plt.close()
else:
    plt.show()

In [ ]:
#fugi = reload(fugi)
#fig=fugi.plotgo_single(param_matrix_cos,xgo_cos,ygo_cos,name="b1map_MF_"+en+"_theory")
#fig.show()

#fugi = reload(fugi)
fig,ax= plt.subplots(1, figsize=(12, 10))
#cs,ax=fugi.plot_interpolation(xgo_uniq,ygo_uniq,param_matrix_MF[:,0,0].reshape(len(xgo_uniq),-1).T*100, ax=ax, cmap=cmap_temp)
cs,ax=fugi.plot_interpolation(np.array(xgo_phi).reshape(12,6),np.array(ygo_phi).reshape(12,6),param_matrix_MF[:,0,0].reshape(12,6)*100, ax=ax, cmap=cmap_temp)

#cs=ax.tricontourf(xgo_cos,ygo_cos,fugi.smoothgauss(param_matrix_phi[:,0,0],0.3,0.6), extend="both",cmap=cmap_temp,levels=20)

ax.set_title("b1 map "+en)
aspect=(xgo_uniq.max()/ygo_uniq.max())
ax.set_xlabel('\u03C6$_{ph}$ [DEG]')
ax.set_ylabel('cos\u03D1$_{ph}$ [adm]')
ax.set_aspect(aspect)

#cbar = fugi.clippedcolorbar(cs,shrink=0.9, ax=cs)
cbar = fig.colorbar(cs,shrink=0.9, ax=ax)
#cbar.set_ticks([totMFPAD.min(),totMFPAD.max()])
cbar.ax.set_ylabel('% of asymmetry')

fig.savefig("../PYTHON_graphs/OUTPUTS/Theory/b1map_phi_MF_"+en+"_theory.png", dpi=300, transparent=False, bbox_inches='tight')

In [ ]:
#fugi = reload(fugi)
#limits=[[-1,2,0.25],[-0.5,0.5,0.1]]
#fig=fugi.plotgo_multiple(param_matrix_cos,xgo_cos,ygo_cos,name="b1-6map_MF_"+en+"_theory",limits=limits, tweak=True)
#fig.show()

fig,axes= plt.subplots(6,1, figsize=(10, 40), sharex=True)
#cs=ax.tricontourf(xgo_cos,ygo_cos,fugi.smoothgauss(param_matrix_cos[:,0,0],0.3,0.6),cmap=cmap_temp,levels=20)
#cs=ax.tricontourf(xgo_cos,ygo_cos,fugi.smoothgauss(param_matrix_cos[:,0,0],0.3,0.6), vmin=-0.2, vmax=0.2,cmap=cmap_temp,levels=20)
#cs=ax.pcolormesh(np.array(xgo_cos),np.array(ygo_cos),fugi.smoothgauss(param_matrix_cos[:,0,0],0.3,0.6),cmap=cmap_temp)
fig.suptitle("b1 map "+en)

for ax,i in zip(axes,range(6)):
    #cs,ax=fugi.plot_interpolation(xgo_uniq,ygo_uniq,param_matrix_MF[:,i,0].reshape(len(xgo_uniq),-1).T, ax=ax, cmap=cmap_temp)
    cs,ax=fugi.plot_interpolation(np.array(xgo_phi).reshape(12,6),np.array(ygo_phi_flip).reshape(12,6),param_matrix_MF[:,i,0].reshape(12,6), ax=ax, cmap=cmap_temp)

    aspect=(xgo_uniq.max()/ygo_uniq.max())
    ax.set_ylabel('cos\u03D1$_{ph}$ [adm]')
    ax.set_aspect(aspect)
    cbar = fig.colorbar(cs,shrink=0.9, ax=ax)
    cbar.ax.set_ylabel("asymmetry")
    if i==5:
        ax.set_xlabel('\u03C6$_{ph}$ [DEG]')
fig.tight_layout()

fig.savefig("../PYTHON_graphs/OUTPUTS/Theory/b1-6map_phi_MF_"+en+"_theory.png", dpi=300, transparent=False, bbox_inches='tight')

if noprint:
    plt.close()
else:
    plt.show()

In [ ]:
#INTEGRATED S AND R MFPADs in MF
fig, ax = plt.subplots(2,2,sharex=True,sharey=True,figsize=(12, 18))

ax[0,0].contourf(phiM,cosM,MFPAD_R_phi_sum.reshape(200,100).T,cmap=cmap_temp)
ax[0,0].set_title("MFPAD R MF")
ax[0,0].set_ylabel('cos\u03D1 [adm]')

ax[0,1].contourf(phiM,cosM,MFPAD_S_phi_sum.reshape(200,100).T,cmap=cmap_temp)
ax[0,1].set_title("MFPAD S MF")

ax[1,0].contourf(phiMM,cosMM,MFPAD_R_phi_LF_sum.T,cmap=cmap_temp, levels=20)
ax[1,0].set_title("MFPAD R LF")
ax[1,0].set_ylabel('cos\u03D1 [adm]')
ax[1,0].set_xlabel('\u03C6 [DEG]')

ax[1,1].contourf(phiMM,cosMM,MFPAD_S_phi_LF_sum.T,cmap=cmap_temp,vmin=0.08,vmax=0.175, levels=20)
ax[1,1].set_title("MFPAD S LF")
ax[1,1].set_xlabel('\u03C6 [DEG]')

#ax[2,0].contourf(phiMM,cosMM,MFPAD_R_cos_LF_sum.T, cmap=cmap_temp, levels=20)
#ax[2,0].set_title("MFPAD S LF norm")
#ax[2,0].set_ylabel('cos\u03D1 [adm]')
#ax[2,0].set_xlabel('\u03C6 [DEG]')

#ax[2,1].contourf(phiMM,cosMM,MFPAD_S_cos_LF_sum.T, cmap=cmap_temp,vmin=4.7, vmax=8.3, levels=20)
#ax[2,1].set_title("MFPAD S LF norm")
#ax[2,1].set_xlabel('\u03C6 [DEG]')
plt.show()

In [ ]:
fig,ax= plt.subplots(1, figsize=(12, 10))
#cs=ax.contourf(phiM,cosM,MFPAD_R_cos_sum.reshape(200,100).T,cmap=cmap_temp)
#cs=ax.contourf(phiMM,cosMM,MFPAD_R_cos_sum.reshape.T, cmap=cmap_temp)
cs,ax=fugi.plot_interpolation(phiM,cosM,MFPAD_R_phi_sum.T, ax=ax, cmap=cmap_temp)
contrast=cs.get_array().max()/cs.get_array().min()

aspect=(phiM.max()/cosM.max())

ax.set_title("Total MFPAD R "+en)
ax.set_xlabel('\u03C6 [DEG]')
ax.set_ylabel('cos\u03D1 [adm]')
cbar = fig.colorbar(cs,shrink=0.9, ax=ax)
#cbar for contourf
#cbar = fugi.clippedcolorbar(cs,shrink=0.9,ax=ax)
cbar.set_ticks([MFPAD_R_phi_sum.min(),MFPAD_R_phi_sum.max()])
cbar.set_ticklabels([1,f"{contrast:.2f}"])
cbar.ax.set_ylabel('contrast')
# cbar.ax.set_ylabel('normalised counts')
ax.set_aspect(aspect)

fig.savefig("../PYTHON_graphs/OUTPUTS/Theory/MFPAD_tot_R_cmap_"+en+"_theory.png", dpi=300, transparent=False, bbox_inches='tight')

In [ ]:
%%capture
fig,ax= plt.subplots(1, figsize=(12, 10))
#cs=ax.contourf(phiM,cosM,MFPAD_R_cos_sum.reshape(200,100).T,cmap=cmap_temp)
#cs=ax.contourf(phiMM,cosMM,MFPAD_R_cos_sum.reshape.T, cmap=cmap_temp)
cs,ax=fugi.plot_interpolation(phiM,cosM,MFPAD_R_phi_sum.T, ax=ax)
contrast=cs.get_array().max()/cs.get_array().min()

aspect=(phiM.max()/cosM.max())

ax.set_title("Total MFPAD R "+en)
ax.set_xlabel('\u03C6 [DEG]')
ax.set_ylabel('cos\u03D1 [adm]')
cbar = fig.colorbar(cs,shrink=0.9, ax=ax)
#cbar for contourf
#cbar = fugi.clippedcolorbar(cs,shrink=0.9,ax=ax)
cbar.set_ticks([MFPAD_R_phi_sum.min(),MFPAD_R_phi_sum.max()])
cbar.set_ticklabels([1,f"{contrast:.2f}"])
cbar.ax.set_ylabel('contrast')
# cbar.ax.set_ylabel('normalised counts')
ax.set_aspect(aspect)

fig.savefig("../PYTHON_graphs/OUTPUTS/Theory/MFPAD_tot_R_"+en+"_theory.png", dpi=300, transparent=False, bbox_inches='tight')

In [ ]:
fig,ax= plt.subplots(1, figsize=(12, 10))
#cs=ax.contourf(phiM,cosM,MFPAD_S_cos_sum.reshape(200,100).T, cmap=cmap_temp)
#cs=ax.contourf(phiMM,cosMM,MFPAD_R_cos_sum.reshape.T, cmap=cmap_temp)
cs,ax=fugi.plot_interpolation(phiM,cosM,MFPAD_S_phi_sum.T, ax=ax, cmap=cmap_temp)
contrast=cs.get_array().max()/cs.get_array().min()

aspect=(phiM.max()/cosM.max())

ax.set_title("Total MFPAD S "+en)
ax.set_xlabel('\u03C6 [DEG]')
ax.set_ylabel('cos\u03D1 [adm]')
cbar = fig.colorbar(cs,shrink=0.9, ax=ax)
#c bar for contourf
#cbar = fugi.clippedcolorbar(cs,shrink=0.9,ax=ax)
cbar.set_ticks([MFPAD_S_phi_sum.min(),MFPAD_S_phi_sum.max()])
cbar.set_ticklabels([1,f"{contrast:.2f}"])
cbar.ax.set_ylabel('contrast')
# cbar.ax.set_ylabel('normalised counts')
ax.set_aspect(aspect)

fig.savefig("../PYTHON_graphs/OUTPUTS/Theory/MFPAD_tot_S_cmap_"+en+"_theory.png", dpi=300, transparent=False, bbox_inches='tight')

In [ ]:
%%capture
fig,ax= plt.subplots(1, figsize=(12, 10))
#cs=ax.contourf(phiM,cosM,MFPAD_S_cos_sum.reshape(200,100).T, cmap=cmap_temp)
#cs=ax.contourf(phiMM,cosMM,MFPAD_R_cos_sum.reshape.T, cmap=cmap_temp)
cs,ax=fugi.plot_interpolation(phiM,cosM,MFPAD_S_phi_sum.T, ax=ax)
contrast=cs.get_array().max()/cs.get_array().min()

aspect=(phiM.max()/cosM.max())

ax.set_title("Total MFPAD S "+en)
ax.set_xlabel('\u03C6 [DEG]')
ax.set_ylabel('cos\u03D1 [adm]')
cbar = fig.colorbar(cs,shrink=0.9, ax=ax)
#c bar for contourf
#cbar = fugi.clippedcolorbar(cs,shrink=0.9,ax=ax)
cbar.set_ticks([MFPAD_S_phi_sum.min(),MFPAD_S_phi_sum.max()])
cbar.set_ticklabels([1,f"{contrast:.2f}"])
cbar.ax.set_ylabel('contrast')
# cbar.ax.set_ylabel('normalised counts')
ax.set_aspect(aspect)

fig.savefig("../PYTHON_graphs/OUTPUTS/Theory/MFPAD_tot_S_"+en+"_theory.png", dpi=300, transparent=False, bbox_inches='tight')

In [ ]:
fig,ax= plt.subplots(1, figsize=(12, 10))
#cs=ax.contourf(phiMM,cosMM,mPECD_cos_sum.T, vmin=-0.6, vmax=0.6,cmap=cmap_temp)
cs,ax=fugi.plot_interpolation(phiM,cosM,mPECD_phi_sum.T*100, ax=ax, cmap=cmap_temp)

aspect=(phiMM.max()/cosMM.max())
ax.set_title("contrast map "+en)
ax.set_xlabel('\u03C6 [DEG]')
ax.set_ylabel('cos\u03D1 [adm]')
cbar=fig.colorbar(cs, shrink=0.9, ax=ax)
#cbar for contourf
#cbar = fugi.clippedcolorbar(cs,shrink=0.9,ax=ax) #colorbar get messed up

cbar.ax.set_ylabel('% of asymmetry')

ax.set_aspect(aspect)

fig.savefig("../PYTHON_graphs/OUTPUTS/Theory/contrastmap_MF_"+en+"_theory.png", dpi=300, transparent=False, bbox_inches='tight')

In [ ]:
#fugi = reload(fugi)
fig,ax= plt.subplots(1, figsize=(12, 10))
#cs=ax.contourf(phiM,cosM,MFPAD_R_cos_sum.reshape(200,100).T,cmap=cmap_temp)
#cs=ax.contourf(phiMM,cosMM,MFPAD_R_cos_sum.reshape.T, cmap=cmap_temp)
limits=[0.07,0.25]
cs,ax=fugi.plot_interpolation(phiM,cosM,MFPAD_R_phi_rot_sum.T, ax=ax, limits=limits)
contrast=cs.get_array().max()/cs.get_array().min()

aspect=(phiM.max()/cosM.max())

ax.set_title("Total MFPAD R opt "+en)
ax.set_xlabel('\u03C6 [DEG]')
ax.set_ylabel('cos\u03D1 [adm]')
cbar = fig.colorbar(cs,shrink=0.9, ax=ax)
#cbar for contourf
#cbar = fugi.clippedcolorbar(cs,shrink=0.9,ax=ax)
cbar.set_ticks([limits[0],limits[1]])
cbar.set_ticklabels([1,f"{contrast:.2f}"])
cbar.ax.set_ylabel('contrast')
# cbar.ax.set_ylabel('normalised counts')
ax.set_aspect(aspect)

fig.savefig("../PYTHON_graphs/OUTPUTS/Theory/MFPAD_tot_R_opt_"+CH+"_MF_"+en+"_theory.png", dpi=300, transparent=False, bbox_inches='tight')

In [ ]:
fig,ax= plt.subplots(1, figsize=(12, 10))
#cs=ax.contourf(phiM,cosM,MFPAD_S_cos_sum.reshape(200,100).T, cmap=cmap_temp)
#cs=ax.contourf(phiMM,cosMM,MFPAD_R_cos_sum.reshape.T, cmap=cmap_temp)
limits=[0.07,0.25]
cs,ax=fugi.plot_interpolation(phiM,cosM,MFPAD_S_phi_rot_sum.T, ax=ax, limits=limits)
contrast=cs.get_array().max()/cs.get_array().min()

aspect=(phiM.max()/cosM.max())

ax.set_title("Total MFPAD S opt "+en)
ax.set_xlabel('\u03C6 [DEG]')
ax.set_ylabel('cos\u03D1 [adm]')
cbar = fig.colorbar(cs,shrink=0.9, ax=ax)
#c bar for contourf
#cbar = fugi.clippedcolorbar(cs,shrink=0.9,ax=ax)
cbar.set_ticks([limits[0],limits[1]])
cbar.set_ticklabels([1,f"{contrast:.2f}"])
cbar.ax.set_ylabel('contrast')
# cbar.ax.set_ylabel('normalised counts')
ax.set_aspect(aspect)

fig.savefig("../PYTHON_graphs/OUTPUTS/Theory/MFPAD_tot_S_opt_"+CH+"_MF_"+en+"_theory.png", dpi=300, transparent=False, bbox_inches='tight')

In [ ]:
#fugi = reload(fugi)
fig,ax= plt.subplots(1, figsize=(12, 10))
#cs=ax.contourf(phiMM,cosMM,mPECD_cos_sum.T, vmin=-0.6, vmax=0.6,cmap=cmap_temp)
cs,ax=fugi.plot_interpolation(phiM,cosM,np.nan_to_num(mPECD_phi_rot_sum).T*100, ax=ax, cmap=cmap_temp, limits=[-45.,45.])

aspect=(phiM.max()/cosM.max())
ax.set_title("contrast map opt "+en)
ax.set_xlabel('\u03C6 [DEG]')
ax.set_ylabel('cos\u03D1 [adm]')
cbar=fig.colorbar(cs, shrink=0.9, ax=ax)
#cbar for contourf
#cbar = fugi.clippedcolorbar(cs,shrink=0.9,ax=ax) #colorbar get messed up
cbar.ax.set_ylabel('% of asymmetry')

ax.set_aspect(aspect)

fig.savefig("../PYTHON_graphs/OUTPUTS/Theory/contrastmap_opt_"+CH+"_MF_"+en+"_theory.png", dpi=300, transparent=False, bbox_inches='tight')

In [ ]:
fig,ax= plt.subplots(1, figsize=(12, 10))
#cs=ax.contourf(phiMM,cosMM,mPECD_LF_cos_sum.T, vmin=-0.8, vmax=0.8,cmap=cmap_temp)
cs,ax=fugi.plot_interpolation(phiM,cosM,mPECD_LF_phi_sum.T*100, ax=ax, cmap=cmap_temp, limits=[-15.,15.])

aspect=(phiM.max()/cosM.max())
ax.set_title("contrast map LF "+en)
ax.set_xlabel('\u03C6 [DEG]')
ax.set_ylabel('cos\u03D1 [adm]')
cbar=fig.colorbar(cs, shrink=0.9, ax=ax)
#cbar for contourf
#cbar = fugi.clippedcolorbar(cs,shrink=0.9,ax=ax) #colorbar get messed up
cbar.ax.set_ylabel('% of asymmetry')

ax.set_aspect(aspect)

fig.savefig("../PYTHON_graphs/OUTPUTS/Theory/contrastmap_LF_"+en+"_theory.png", dpi=300, transparent=False, bbox_inches='tight')

In [ ]:
from matplotlib.font_manager import findfont, FontProperties
font = findfont(FontProperties(family=['sans-serif']))
font